## load the pre-saved nodes and edges to check the CVE information

In [3]:
import sys
from pathlib import Path
sys.path.insert(0, Path(sys.path[0]).parent.as_posix())
from lxml import etree
import pickle
from cent import between_cent, degree_cent, eigen_cent
import logging
from pathlib import Path
from datetime import datetime

logging.basicConfig(level=logging.DEBUG,
                    format='%(asctime)s [%(levelname)s]: %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S'
                )
logger = logging.getLogger(__name__)
file_handler = logging.FileHandler('cent_cal.log')
file_handler.setLevel(logging.DEBUG)
file_formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
file_handler.setFormatter(file_formatter)
logger.addHandler(file_handler)

In [2]:
!pip3 install dask

In [16]:
import dask.dataframe as dd

cve_path = Path.cwd().parent.joinpath("data", 'cve_data.csv')
cve_df = dd.read_csv(cve_path.as_posix())
cve_df = cve_df.compute()
for column in cve_df.columns:
    print(column, cve_df[column].value_counts())


2024-11-28 20:24:03 [DEBUG]: open file: /nfs/primary/MSRCall/DPAnalyzer/data/cve_data.csv


source source
n18741       4640
n1597975     3615
n347688      3569
n4307        3282
n5524        3159
             ... 
n12724654       1
n12723323       1
n12737959       1
n12737958       1
n15117230       1
Name: count, Length: 15117217, dtype: int64
target target
n1102961     2776154
n771170      1966436
n83995       1716315
n380430      1578793
n5202385     1098582
              ...   
n20132263          1
n20132264          1
n20132265          1
n20132266          1
n59152711          1
Name: count, Length: 58807430, dtype: int64
source_cve_exists source_cve_exists
0    176903368
1      1251672
Name: count, dtype: int64
source_cve_score source_cve_score
0      176903403
3         396132
4         213765
2         210867
5          77281
         ...    
64             9
48             6
81             6
102            4
71             3
Name: count, Length: 134, dtype: int64
source_cve_num source_cve_num
0     176903368
1        805404
2        212710
3         70606
4        

In [17]:
cve_sib_path = Path.cwd().parent.joinpath("data", 'cve_2_siblings_data.csv')
cve_sib_df = dd.read_csv(cve_path.as_posix())
cve_sib_df = cve_sib_df.compute()
for column in cve_sib_df.columns:
    if column not in ['source', 'target']:
        print(column, cve_sib_df[column].value_counts())

2024-11-28 20:26:33 [DEBUG]: open file: /nfs/primary/MSRCall/DPAnalyzer/data/cve_data.csv


source_cve_exists source_cve_exists
0    176903368
1      1251672
Name: count, dtype: int64
source_cve_score source_cve_score
0      176903403
3         396132
4         213765
2         210867
5          77281
         ...    
64             9
48             6
81             6
102            4
71             3
Name: count, Length: 134, dtype: int64
source_cve_num source_cve_num
0     176903368
1        805404
2        212710
3         70606
4         46544
        ...    
64            9
65            9
56            9
60            9
51            9
Name: count, Length: 61, dtype: int64
target_cve_exists target_cve_exists
0    178077647
1        77393
Name: count, dtype: int64
target_cve_score target_cve_score
0      178077656
3          22289
2          14933
4          13721
5           3739
         ...    
181            1
197            1
214            1
210            1
71             1
Name: count, Length: 134, dtype: int64
target_cve_num target_cve_num
0     178077647
1     

In [4]:
def save_data(nodes, edges, file_path):
    with file_path.open('wb') as f:
        pickle.dump({'nodes': nodes, 'edges': edges}, f)        
    

def load_data(file_path):
    with file_path.open('rb') as f:
        data = pickle.load(f)
    return data['nodes'], data['edges']

In [5]:
graph_path = Path.cwd().parent.joinpath("data", 'graph_nodes_edges.pkl')

if  graph_path.exists():
    logger.info("Loading nodes and edges from saved file.")
    nodes, edges = load_data(graph_path)
else:
    file_path = Path.cwd().parent.joinpath("data", "graph_metric.graphml").as_posix()
    logger.info("Parsing nodes and edges from GraphML.")
    # generate nodes and edges from graphml
    now = datetime.now()
    nodes, edges = parse_graphml_in_chunks(file_path)
    logger.info(f"Time spent for node loading from graphml is: {datetime.now() - now}")
    save_data(nodes, edges, graph_path)


2024-12-01 10:58:33 [INFO]: Loading nodes and edges from saved file.


In [7]:
addvalue_dict = {'n14990848': ['n59148121'], 'n14965429': ['n59148122'], 'n14951273': ['n59148123'], 'n14953696': ['n59148124'], 'n14975461': ['n59148125'], 'n14973043': ['n59148126'], 'n14954781': ['n59148127'], 'n14998940': ['n59148128'], 'n14976911': ['n59148129'], 'n14951857': ['n59148130'], 'n14974976': ['n59148131'], 'n14992160': ['n59148132'], 'n14979190': ['n59148133'], 'n14983202': ['n59148134'], 'n14968056': ['n59148135'], 'n14981267': ['n59148136'], 'n14962990': ['n59148137'], 'n14962500': ['n59148138'], 'n14982363': ['n59148139'], 'n14963835': ['n59148140'], 'n14980181': ['n59148141'], 'n14962745': ['n59148142'], 'n14986959': ['n59148143'], 'n14996758': ['n59148144'], 'n14959108': ['n59148145'], 'n14993007': ['n59148146'], 'n14995427': ['n59148147'], 'n14993486': ['n59148148'], 'n14998933': ['n59148149'], 'n14972560': ['n59148150'], 'n14959586': ['n59148151'], 'n14970363': ['n59148152'], 'n14974741': ['n59148153'], 'n14950757': ['n59148154'], 'n14972567': ['n59148155'], 'n14950755': ['n59148156'], 'n14950758': ['n59148157'], 'n14951605': ['n59148158'], 'n14967199': ['n59148159'], 'n14982123': ['n59148160'], 'n14986972': ['n59148161'], 'n14958484': ['n59148162'], 'n14960310': ['n59148163'], 'n14981284': ['n59148164'], 'n14987811': ['n59148165'], 'n14962730': ['n59148166'], 'n14981283': ['n59148167'], 'n14971209': ['n59148168'], 'n14962976': ['n59148169'], 'n14988909': ['n59148170'], 'n14994585': ['n59148171'], 'n14998941': ['n59148172'], 'n14970395': ['n59148173'], 'n14968076': ['n59148174'], 'n14974952': ['n59148175'], 'n14985609': ['n59148176'], 'n14972778': ['n59148177'], 'n14983421': ['n59148178'], 'n14959362': ['n59148179'], 'n14964947': ['n59148180'], 'n14986935': ['n59148181'], 'n14996739': ['n59148182'], 'n14994557': ['n59148183'], 'n14994314': ['n59148184'], 'n14964707': ['n59148185'], 'n14995400': ['n59148186'], 'n14970144': ['n59148187'], 'n14992134': ['n59148188'], 'n14992377': ['n59148189'], 'n14995884': ['n59148190'], 'n14970380': ['n59148191'], 'n14972303': ['n59148192'], 'n14972304': ['n59148193'], 'n14992390': ['n59148194'], 'n14985613': ['n59148195'], 'n14981013': ['n59148196'], 'n14981255': ['n59148197'], 'n14960582': ['n59148198'], 'n14981254': ['n59148199'], 'n14962996': ['n59148200'], 'n14959592': ['n59148201'], 'n14962758': ['n59148202'], 'n14980170': ['n59148203'], 'n14956081': ['n59148204'], 'n14986709': ['n59148205'], 'n14998923': ['n59148206'], 'n14950780': ['n59148207'], 'n14956088': ['n59148208'], 'n14993476': ['n59148209'], 'n14996500': ['n59148210'], 'n14952745': ['n59148211'], 'n14970175': ['n59148212'], 'n14950323': ['n59148213'], 'n14995233': ['n59148214'], 'n14997892': ['n59148215'], 'n14972115': ['n59148216'], 'n14994380': ['n59148217'], 'n14950326': ['n59148218'], 'n14950568': ['n59148219'], 'n14950325': ['n59148220'], 'n14975627': ['n59148221'], 'n14982399': ['n59148222'], 'n14954927': ['n59148223'], 'n14984100': ['n59148224'], 'n14962308': ['n59148225'], 'n14962309': ['n59148226'], 'n14976961': ['n59148227'], 'n14994379': ['n59148228'], 'n14950570': ['n59148229'], 'n14971012': ['n59148230'], 'n14994378': ['n59148231'], 'n14952979': ['n59148232'], 'n14985439': ['n59148233'], 'n14978908': ['n59148234'], 'n14984105': ['n59148235'], 'n14962781': ['n59148236'], 'n14961687': ['n59148237'], 'n14988945': ['n59148238'], 'n14959131': ['n59148239'], 'n14963863': ['n59148240'], 'n14986525': ['n59148241'], 'n14988944': ['n59148242'], 'n14962537': ['n59148243'], 'n14961446': ['n59148244'], 'n14984590': ['n59148245'], 'n14997418': ['n59148246'], 'n14952982': ['n59148247'], 'n14974790': ['n59148248'], 'n14993056': ['n59148249'], 'n14998987': ['n59148250'], 'n14975400': ['n59148251'], 'n14994147': ['n59148252'], 'n14994120': ['n59148253'], 'n14998720': ['n59148254'], 'n14996782': ['n59148255'], 'n14973667': ['n59148256'], 'n14954944': ['n59148257'], 'n14983225': ['n59148258'], 'n14960396': ['n59148259'], 'n14964742': ['n59148260'], 'n14987823': ['n59148261'], 'n14987826': ['n59148262'], 'n14953860': ['n59148263'], 'n14953861': ['n59148264'], 'n14995444': ['n59148265'], 'n14997623': ['n59148266'], 'n14971278': ['n59148267'], 'n14997624': ['n59148268'], 'n14996794': ['n59148269'], 'n14972102': ['n59148270'], 'n14950578': ['n59148271'], 'n14953602': ['n59148272'], 'n14984325': ['n59148273'], 'n14964976': ['n59148274'], 'n14986984': ['n59148275'], 'n14960133': ['n59148276'], 'n14983485': ['n59148277'], 'n14986501': ['n59148278'], 'n14962311': ['n59148279'], 'n14959150': ['n59148280'], 'n14962557': ['n59148281'], 'n14964977': ['n59148282'], 'n14972590': ['n59148283'], 'n14972593': ['n59148284'], 'n14950340': ['n59148285'], 'n14973441': ['n59148286'], 'n14950581': ['n59148287'], 'n14973200': ['n59148288'], 'n14968570': ['n59148289'], 'n14969660': ['n59148290'], 'n14980216': ['n59148291'], 'n14982637': ['n59148292'], 'n14968331': ['n59148293'], 'n14989037': ['n59148294'], 'n14958770': ['n59148295'], 'n14979473': ['n59148296'], 'n14991108': ['n59148297'], 'n14978388': ['n59148298'], 'n14955267': ['n59148299'], 'n14958537': ['n59148300'], 'n14960603': ['n59148301'], 'n14978387': ['n59148302'], 'n14994613': ['n59148303'], 'n14979239': ['n59148304'], 'n14990015': ['n59148305'], 'n14991583': ['n59148306'], 'n14953088': ['n59148307'], 'n14971990': ['n59148308'], 'n14958518': ['n59148309'], 'n14982649': ['n59148310'], 'n14989285': ['n59148311'], 'n14965058': ['n59148312'], 'n14979480': ['n59148313'], 'n14994868': ['n59148314'], 'n14994628': ['n59148315'], 'n14979486': ['n59148316'], 'n14979485': ['n59148317'], 'n14955252': ['n59148318'], 'n14954165': ['n59148319'], 'n14966171': ['n59148320'], 'n14965080': ['n59148321'], 'n14983702': ['n59148322'], 'n14971727': ['n59148323'], 'n14981761': ['n59148324'], 'n14968595': ['n59148325'], 'n14988168': ['n59148326'], 'n14968355': ['n59148327'], 'n14994839': ['n59148328'], 'n14969439': ['n59148329'], 'n14960626': ['n59148330'], 'n14975096': ['n59148331'], 'n14957467': ['n59148332'], 'n14977032': ['n59148333'], 'n14990230': ['n59148334'], 'n14992655': ['n59148335'], 'n14961716': ['n59148336'], 'n14970663': ['n59148337'], 'n14971510': ['n59148338'], 'n14988170': ['n59148339'], 'n14971731': ['n59148340'], 'n14982627': ['n59148341'], 'n14951900': ['n59148342'], 'n14989260': ['n59148343'], 'n14965071': ['n59148344'], 'n14982621': ['n59148345'], 'n14982622': ['n59148346'], 'n14972828': ['n59148347'], 'n14967014': ['n59148348'], 'n14954181': ['n59148349'], 'n14994605': ['n59148350'], 'n14961941': ['n59148351'], 'n14961702': ['n59148352'], 'n14958788': ['n59148353'], 'n14953097': ['n59148354'], 'n14970893': ['n59148355'], 'n14958784': ['n59148356'], 'n14990244': ['n59148357'], 'n14967285': ['n59148358'], 'n14974812': ['n59148359'], 'n14974811': ['n59148360'], 'n14959419': ['n59148361'], 'n14975903': ['n59148362'], 'n14951937': ['n59148363'], 'n14975904': ['n59148364'], 'n14980256': ['n59148365'], 'n14973728': ['n59148366'], 'n14985944': ['n59148367'], 'n14980255': ['n59148368'], 'n14968377': ['n59148369'], 'n14981342': ['n59148370'], 'n14957484': ['n59148371'], 'n14979279': ['n59148372'], 'n14994656': ['n59148373'], 'n14958339': ['n59148374'], 'n14970683': ['n59148375'], 'n14971791': ['n59148376'], 'n14991144': ['n59148377'], 'n14993322': ['n59148378'], 'n14991161': ['n59148379'], 'n14973731': ['n59148380'], 'n14967038': ['n59148381'], 'n14980025': ['n59148382'], 'n14979280': ['n59148383'], 'n14980031': ['n59148384'], 'n14959890': ['n59148385'], 'n14956149': ['n59148386'], 'n14958325': ['n59148387'], 'n14990065': ['n59148388'], 'n14961727': ['n59148389'], 'n14994421': ['n59148390'], 'n14997932': ['n59148391'], 'n14995993': ['n59148392'], 'n14992487': ['n59148393'], 'n14995970': ['n59148394'], 'n14969000': ['n59148395'], 'n14984838': ['n59148396'], 'n14951954': ['n59148397'], 'n14972855': ['n59148398'], 'n14951711': ['n59148399'], 'n14968391': ['n59148400'], 'n14985923': ['n59148401'], 'n14980479': ['n59148402'], 'n14967066': ['n59148403'], 'n14968158': ['n59148404'], 'n14956174': ['n59148405'], 'n14959684': ['n59148406'], 'n14963931': ['n59148407'], 'n14960668': ['n59148408'], 'n14992219': ['n59148409'], 'n14979014': ['n59148410'], 'n14979498': ['n59148411'], 'n14994875': ['n59148412'], 'n14990293': ['n59148413'], 'n14950612': ['n59148414'], 'n14973713': ['n59148415'], 'n14971778': ['n59148416'], 'n14951706': ['n59148417'], 'n14972627': ['n59148418'], 'n14950616': ['n59148419'], 'n14957010': ['n59148420'], 'n14981581': ['n59148421'], 'n14959672': ['n59148422'], 'n14962833': ['n59148423'], 'n14994408': ['n59148424'], 'n14990286': ['n59148425'], 'n14994402': ['n59148426'], 'n14970698': ['n59148427'], 'n14979028': ['n59148428'], 'n14980617': ['n59148429'], 'n14980619': ['n59148430'], 'n14998387': ['n59148431'], 'n14955656': ['n59148432'], 'n14956748': ['n59148433'], 'n14963040': ['n59148434'], 'n14988344': ['n59148435'], 'n14963044': ['n59148436'], 'n14986169': ['n59148437'], 'n14992837': ['n59148438'], 'n14993922': ['n59148439'], 'n14957841': ['n59148440'], 'n14973096': ['n59148441'], 'n14957840': ['n59148442'], 'n14979639': ['n59148443'], 'n14982808': ['n59148444'], 'n14998158': ['n59148445'], 'n14981959': ['n59148446'], 'n14999487': ['n59148447'], 'n14954799': ['n59148448'], 'n14980867': ['n59148449'], 'n14962185': ['n59148450'], 'n14980861': ['n59148451'], 'n14963271': ['n59148452'], 'n14961095': ['n59148453'], 'n14966303': ['n59148454'], 'n14966545': ['n59148455'], 'n14963034': ['n59148456'], 'n14971918': ['n59148457'], 'n14966777': ['n59148458'], 'n14979880': ['n59148459'], 'n14992602': ['n59148460'], 'n14965209': ['n59148461'], 'n14968958': ['n59148462'], 'n14968957': ['n59148463'], 'n14952385': ['n59148464'], 'n14976136': ['n59148465'], 'n14954562': ['n59148466'], 'n14975009': ['n59148467'], 'n14988563': ['n59148468'], 'n14965247': ['n59148469'], 'n14966571': ['n59148470'], 'n14969840': ['n59148471'], 'n14970826': ['n59148472'], 'n14966568': ['n59148473'], 'n14993901': ['n59148474'], 'n14966569': ['n59148475'], 'n14967658': ['n59148476'], 'n14976581': ['n59148477'], 'n14958710': ['n59148478'], 'n14955684': ['n59148479'], 'n14997027': ['n59148480'], 'n14979852': ['n59148481'], 'n14999467': ['n59148482'], 'n14953248': ['n59148483'], 'n14999463': ['n59148484'], 'n14979619': ['n59148485'], 'n14956757': ['n59148486'], 'n14967896': ['n59148487'], 'n14964383': ['n59148488'], 'n14963052': ['n59148489'], 'n14964389': ['n59148490'], 'n14968980': ['n59148491'], 'n14968741': ['n59148492'], 'n14965475': ['n59148493'], 'n14973083': ['n59148494'], 'n14979862': ['n59148495'], 'n14973084': ['n59148496'], 'n14985060': ['n59148497'], 'n14953492': ['n59148498'], 'n14974179': ['n59148499'], 'n14990640': ['n59148500'], 'n14983927': ['n59148501'], 'n14954128': ['n59148502'], 'n14966355': ['n59148503'], 'n14987297': ['n59148504'], 'n14987298': ['n59148505'], 'n14987059': ['n59148506'], 'n14967685': ['n59148507'], 'n14965259': ['n59148508'], 'n14992878': ['n59148509'], 'n14993720': ['n59148510'], 'n14955468': ['n59148511'], 'n14954378': ['n59148512'], 'n14956313': ['n59148513'], 'n14959821': ['n59148514'], 'n14979679': ['n59148515'], 'n14978589': ['n59148516'], 'n14953285': ['n59148517'], 'n14977498': ['n59148518'], 'n14983939': ['n59148519'], 'n14978107': ['n59148520'], 'n14959806': ['n59148521'], 'n14957628': ['n59148522'], 'n14982603': ['n59148523'], 'n14982845': ['n59148524'], 'n14989001': ['n59148525'], 'n14989484': ['n59148526'], 'n14966347': ['n59148527'], 'n14952181': ['n59148528'], 'n14969605': ['n59148529'], 'n14967427': ['n59148530'], 'n14975085': ['n59148531'], 'n14979684': ['n59148532'], 'n14975087': ['n59148533'], 'n14990466': ['n59148534'], 'n14952185': ['n59148535'], 'n14978114': ['n59148536'], 'n14991310': ['n59148537'], 'n14980634': ['n59148538'], 'n14981724': ['n59148539'], 'n14988360': ['n59148540'], 'n14997073': ['n59148541'], 'n14989696': ['n59148542'], 'n14988364': ['n59148543'], 'n14967220': ['n59148544'], 'n14968312': ['n59148545'], 'n14955482': ['n59148546'], 'n14970627': ['n59148547'], 'n14991766': ['n59148548'], 'n14992613': ['n59148549'], 'n14976382': ['n59148550'], 'n14976141': ['n59148551'], 'n14958994': ['n59148552'], 'n14999269': ['n59148553'], 'n14965272': ['n59148554'], 'n14965270': ['n59148555'], 'n14982823': ['n59148556'], 'n14988130': ['n59148557'], 'n14963098': ['n59148558'], 'n14982822': ['n59148559'], 'n14967458': ['n59148560'], 'n14965036': ['n59148561'], 'n14970615': ['n59148562'], 'n14992626': ['n59148563'], 'n14977243': ['n59148564'], 'n14977483': ['n59148565'], 'n14953059': ['n59148566'], 'n14979661': ['n59148567'], 'n14991534': ['n59148568'], 'n14955231': ['n59148569'], 'n14956322': ['n59148570'], 'n14978577': ['n59148571'], 'n14975069': ['n59148572'], 'n15026368': ['n59148573'], 'n15026124': ['n59148574'], 'n15027697': ['n59148575'], 'n15023099': ['n59148576'], 'n15029631': ['n59148577'], 'n15002155': ['n59148578'], 'n15037021': ['n59148579'], 'n15006999': ['n59148580'], 'n15007844': ['n59148581'], 'n15018739': ['n59148582'], 'n15014137': ['n59148583'], 'n15014379': ['n59148584'], 'n15042618': ['n59148585'], 'n15015466': ['n59148586'], 'n15038344': ['n59148587'], 'n15013043': ['n59148588'], 'n15012197': ['n59148589'], 'n15014132': ['n59148590'], 'n15028310': ['n59148591'], 'n15016553': ['n59148592'], 'n15029882': ['n59148593'], 'n15029881': ['n59148594'], 'n15017403': ['n59148595'], 'n15049012': ['n59148596'], 'n15032801': ['n59148597'], 'n15027228': ['n59148598'], 'n15048387': ['n59148599'], 'n15001057': ['n59148600'], 'n15005419': ['n59148601'], 'n15033904': ['n59148602'], 'n15021921': ['n59148603'], 'n15042842': ['n59148604'], 'n15005898': ['n59148605'], 'n15042843': ['n59148606'], 'n15043932': ['n59148607'], 'n15038360': ['n59148608'], 'n15041998': ['n59148609'], 'n15042606': ['n59148610'], 'n15011097': ['n59148611'], 'n15037025': ['n59148612'], 'n15030874': ['n59148613'], 'n15017875': ['n59148614'], 'n15018965': ['n59148615'], 'n15026381': ['n59148616'], 'n15015219': ['n59148617'], 'n15016789': ['n59148618'], 'n15047062': ['n59148619'], 'n15030649': ['n59148620'], 'n15027479': ['n59148621'], 'n15029416': ['n59148622'], 'n15031737': ['n59148623'], 'n15025056': ['n59148624'], 'n15004796': ['n59148625'], 'n15042632': ['n59148626'], 'n15043964': ['n59148627'], 'n15004311': ['n59148628'], 'n15004556': ['n59148629'], 'n15041545': ['n59148630'], 'n15042876': ['n59148631'], 'n15042877': ['n59148632'], 'n15043723': ['n59148633'], 'n15038371': ['n59148634'], 'n15045908': ['n59148635'], 'n15044818': ['n59148636'], 'n15011087': ['n59148637'], 'n15031731': ['n59148638'], 'n15027241': ['n59148639'], 'n15016776': ['n59148640'], 'n15049033': ['n59148641'], 'n15018956': ['n59148642'], 'n15047096': ['n59148643'], 'n15007817': ['n59148644'], 'n15049019': ['n59148645'], 'n15027486': ['n59148646'], 'n15001273': ['n59148647'], 'n15038384': ['n59148648'], 'n15011081': ['n59148649'], 'n15042620': ['n59148650'], 'n15004301': ['n59148651'], 'n15004543': ['n59148652'], 'n15038385': ['n59148653'], 'n15005877': ['n59148654'], 'n15012168': ['n59148655'], 'n15040453': ['n59148656'], 'n15030891': ['n59148657'], 'n15030655': ['n59148658'], 'n15025071': ['n59148659'], 'n15023057': ['n59148660'], 'n15027416': ['n59148661'], 'n15048113': ['n59148662'], 'n15026566': ['n59148663'], 'n15049200': ['n59148664'], 'n15004375': ['n59148665'], 'n15006795': ['n59148666'], 'n15007885': ['n59148667'], 'n15035284': ['n59148668'], 'n15040629': ['n59148669'], 'n15008735': ['n59148670'], 'n15040625': ['n59148671'], 'n15003292': ['n59148672'], 'n15037210': ['n59148673'], 'n15004382': ['n59148674'], 'n15036365': ['n59148675'], 'n15037697': ['n59148676'], 'n15019622': ['n59148677'], 'n15016359': ['n59148678'], 'n15017206': ['n59148679'], 'n15049451': ['n59148680'], 'n15049450': ['n59148681'], 'n15028758': ['n59148682'], 'n15047013': ['n59148683'], 'n15025246': ['n59148684'], 'n15030835': ['n59148685'], 'n15007639': ['n59148686'], 'n15027424': ['n59148687'], 'n15029603': ['n59148688'], 'n15045075': ['n59148689'], 'n15026578': ['n59148690'], 'n15027667': ['n59148691'], 'n15048105': ['n59148692'], 'n15048346': ['n59148693'], 'n15007633': ['n59148694'], 'n15036142': ['n59148695'], 'n15040633': ['n59148696'], 'n15037228': ['n59148697'], 'n15035285': ['n59148698'], 'n15039400': ['n59148699'], 'n15041718': ['n59148700'], 'n15003284': ['n59148701'], 'n15015497': ['n59148702'], 'n15039644': ['n59148703'], 'n15017433': ['n59148704'], 'n15030830': ['n59148705'], 'n15018767': ['n59148706'], 'n15047287': ['n59148707'], 'n15028521': ['n59148708'], 'n15025013': ['n59148709'], 'n15003264': ['n59148710'], 'n15004597': ['n59148711'], 'n15040408': ['n59148712'], 'n15004357': ['n59148713'], 'n15015490': ['n59148714'], 'n15002180': ['n59148715'], 'n15006540': ['n59148716'], 'n15003272': ['n59148717'], 'n15014154': ['n59148718'], 'n15039419': ['n59148719'], 'n15018517': ['n59148720'], 'n15047036': ['n59148721'], 'n15023088': ['n59148722'], 'n15018992': ['n59148723'], 'n15049699': ['n59148724'], 'n15003253': ['n59148725'], 'n15006765': ['n59148726'], 'n15013064': ['n59148727'], 'n15002166': ['n59148728'], 'n15007852': ['n59148729'], 'n15006766': ['n59148730'], 'n15005679': ['n59148731'], 'n15037491': ['n59148732'], 'n15042825': ['n59148733'], 'n15017419': ['n59148734'], 'n15036159': ['n59148735'], 'n15042826': ['n59148736'], 'n15040897': ['n59148737'], 'n15039422': ['n59148738'], 'n15039664': ['n59148739'], 'n15036155': ['n59148740'], 'n15003263': ['n59148741'], 'n15023093': ['n59148742'], 'n15017411': ['n59148743'], 'n15035917': ['n59148744'], 'n15001904': ['n59148745'], 'n15007204': ['n59148746'], 'n15020424': ['n59148747'], 'n15022604': ['n59148748'], 'n15009623': ['n59148749'], 'n15021753': ['n59148750'], 'n15004183': ['n59148751'], 'n15005033': ['n59148752'], 'n15032644': ['n59148753'], 'n15029486': ['n59148754'], 'n15032403': ['n59148755'], 'n15016158': ['n59148756'], 'n15027061': ['n59148757'], 'n15029009': ['n59148758'], 'n15006346': ['n59148759'], 'n15039299': ['n59148760'], 'n15044866': ['n59148761'], 'n15023700': ['n59148762'], 'n15044867': ['n59148763'], 'n15007681': ['n59148764'], 'n15044629': ['n59148765'], 'n15030471': ['n59148766'], 'n15008772': ['n59148767'], 'n15040277': ['n59148768'], 'n15016145': ['n59148769'], 'n15033503': ['n59148770'], 'n15018569': ['n59148771'], 'n15042470': ['n59148772'], 'n15042471': ['n59148773'], 'n15043322': ['n59148774'], 'n15007424': ['n59148775'], 'n15023714': ['n59148776'], 'n15023959': ['n59148777'], 'n15020206': ['n59148778'], 'n15030483': ['n59148779'], 'n15033992': ['n59148780'], 'n15040045': ['n59148781'], 'n15018313': ['n59148782'], 'n15033514': ['n59148783'], 'n15018556': ['n59148784'], 'n15022630': ['n59148785'], 'n15044880': ['n59148786'], 'n15020694': ['n59148787'], 'n15005234': ['n59148788'], 'n15021789': ['n59148789'], 'n15041377': ['n59148790'], 'n15042224': ['n59148791'], 'n15005478': ['n59148792'], 'n15003058': ['n59148793'], 'n15007412': ['n59148794'], 'n15009831': ['n59148795'], 'n15015041': ['n59148796'], 'n15044889': ['n59148797'], 'n15014191': ['n59148798'], 'n15024813': ['n59148799'], 'n15043319': ['n59148800'], 'n15024812': ['n59148801'], 'n15029270': ['n59148802'], 'n15034852': ['n59148803'], 'n15033520': ['n59148804'], 'n15016124': ['n59148805'], 'n15018543': ['n59148806'], 'n15035700': ['n59148807'], 'n15049048': ['n59148808'], 'n15028106': ['n59148809'], 'n15031518': ['n59148810'], 'n15018144': ['n59148811'], 'n15018143': ['n59148812'], 'n15025079': ['n59148813'], 'n15042895': ['n59148814'], 'n15001703': ['n59148815'], 'n15044837': ['n59148816'], 'n15010891': ['n59148817'], 'n15006166': ['n59148818'], 'n15040222': ['n59148819'], 'n15008342': ['n59148820'], 'n15018137': ['n59148821'], 'n15030421': ['n59148822'], 'n15017048': ['n59148823'], 'n15018378': ['n59148824'], 'n15027265': ['n59148825'], 'n15030420': ['n59148826'], 'n15033931': ['n59148827'], 'n15011989': ['n59148828'], 'n15012834': ['n59148829'], 'n15005070': ['n59148830'], 'n15016193': ['n59148831'], 'n15033708': ['n59148832'], 'n15029444': ['n59148833'], 'n15000844': ['n59148834'], 'n15042645': ['n59148835'], 'n15005061': ['n59148836'], 'n15041319': ['n59148837'], 'n15042409': ['n59148838'], 'n15007244': ['n59148839'], 'n15023905': ['n59148840'], 'n15030672': ['n59148841'], 'n15006154': ['n59148842'], 'n15008573': ['n59148843'], 'n15009420': ['n59148844'], 'n15013916': ['n59148845'], 'n15030432': ['n59148846'], 'n15012824': ['n59148847'], 'n15026184': ['n59148848'], 'n15031762': ['n59148849'], 'n15010648': ['n59148850'], 'n15012825': ['n59148851'], 'n15028362': ['n59148852'], 'n15010885': ['n59148853'], 'n15010642': ['n59148854'], 'n15025093': ['n59148855'], 'n15001923': ['n59148856'], 'n15029458': ['n59148857'], 'n15001926': ['n59148858'], 'n15017035': ['n59148859'], 'n15019696': ['n59148860'], 'n15042673': ['n59148861'], 'n15021978': ['n59148862'], 'n15000833': ['n59148863'], 'n15044616': ['n59148864'], 'n15007472': ['n59148865'], 'n15045708': ['n59148866'], 'n15016178': ['n59148867'], 'n15017267': ['n59148868'], 'n15028134': ['n59148869'], 'n15040242': ['n59148870'], 'n15011969': ['n59148871'], 'n15028133': ['n59148872'], 'n15028131': ['n59148873'], 'n15029461': ['n59148874'], 'n15019200': ['n59148875'], 'n15043995': ['n59148876'], 'n15042422': ['n59148877'], 'n15006124': ['n59148878'], 'n15006125': ['n59148879'], 'n15009875': ['n59148880'], 'n15022830': ['n59148881'], 'n15007217': ['n59148882'], 'n15042669': ['n59148883'], 'n15043759': ['n59148884'], 'n15015081': ['n59148885'], 'n15022833': ['n59148886'], 'n15043758': ['n59148887'], 'n15005281': ['n59148888'], 'n15042429': ['n59148889'], 'n15030690': ['n59148890'], 'n15009643': ['n59148891'], 'n15032871': ['n59148892'], 'n15007462': ['n59148893'], 'n15027297': ['n59148894'], 'n15030212': ['n59148895'], 'n15049063': ['n59148896'], 'n15040252': ['n59148897'], 'n15049062': ['n59148898'], 'n15033721': ['n59148899'], 'n15033726': ['n59148900'], 'n15031789': ['n59148901'], 'n15022565': ['n59148902'], 'n15023653': ['n59148903'], 'n15024500': ['n59148904'], 'n15018069': ['n59148905'], 'n15044356': ['n59148906'], 'n15024980': ['n59148907'], 'n15009348': ['n59148908'], 'n15045447': ['n59148909'], 'n15049809': ['n59148910'], 'n15022569': ['n59148911'], 'n15045209': ['n59148912'], 'n15009110': ['n59148913'], 'n15008266': ['n59148914'], 'n15034548': ['n59148915'], 'n15037817': ['n59148916'], 'n15018059': ['n59148917'], 'n15034305': ['n59148918'], 'n15034549': ['n59148919'], 'n15023421': ['n59148920'], 'n15021486': ['n59148921'], 'n15043493': ['n59148922'], 'n15044340': ['n59148923'], 'n15045434': ['n59148924'], 'n15048940': ['n59148925'], 'n15044347': ['n59148926'], 'n15001617': ['n59148927'], 'n15044588': ['n59148928'], 'n15009336': ['n59148929'], 'n15020399': ['n59148930'], 'n15044109': ['n59148931'], 'n15000765': ['n59148932'], 'n15011890': ['n59148933'], 'n15014928': ['n59148934'], 'n15011419': ['n59148935'], 'n15018283': ['n59148936'], 'n15022342': ['n59148937'], 'n15018046': ['n59148938'], 'n15026708': ['n59148939'], 'n15025617': ['n59148940'], 'n15026709': ['n59148941'], 'n15046799': ['n59148942'], 'n15026707': ['n59148943'], 'n15023678': ['n59148944'], 'n15009571': ['n59148945'], 'n15033476': ['n59148946'], 'n15011408': ['n59148947'], 'n15033479': ['n59148948'], 'n15012977': ['n59148949'], 'n15011885': ['n59148950'], 'n15022596': ['n59148951'], 'n15020172': ['n59148952'], 'n15023687': ['n59148953'], 'n15043274': ['n59148954'], 'n15019365': ['n59148955'], 'n15048965': ['n59148956'], 'n15046303': ['n59148957'], 'n15016099': ['n59148958'], 'n15019125': ['n59148959'], 'n15045695': ['n59148960'], 'n15044124': ['n59148961'], 'n15044129': ['n59148962'], 'n15009314': ['n59148963'], 'n15023207': ['n59148964'], 'n15008229': ['n59148965'], 'n15033484': ['n59148966'], 'n15006057': ['n59148967'], 'n15032157': ['n59148968'], 'n15013815': ['n59148969'], 'n15034337': ['n59148970'], 'n15012968': ['n59148971'], 'n15010544': ['n59148972'], 'n15042374': ['n59148973'], 'n15003608': ['n59148974'], 'n15044553': ['n59148975'], 'n15044558': ['n59148976'], 'n15020586': ['n59148977'], 'n15043468': ['n59148978'], 'n15044559': ['n59148979'], 'n15019193': ['n59148980'], 'n15022767': ['n59148981'], 'n15009155': ['n59148982'], 'n15033891': ['n59148983'], 'n15012310': ['n59148984'], 'n15012552': ['n59148985'], 'n15016915': ['n59148986'], 'n15030147': ['n59148987'], 'n15032329': ['n59148988'], 'n15034749': ['n59148989'], 'n15011226': ['n59148990'], 'n15029161': ['n59148991'], 'n15021440': ['n59148992'], 'n15001899': ['n59148993'], 'n15003836': ['n59148994'], 'n15035609': ['n59148995'], 'n15044540': ['n59148996'], 'n15044782': ['n59148997'], 'n15044545': ['n59148998'], 'n15024957': ['n59148999'], 'n15042367': ['n59149000'], 'n15045875': ['n59149001'], 'n15046964': ['n59149002'], 'n15041038': ['n59149003'], 'n15045634': ['n59149004'], 'n15021687': ['n59149005'], 'n15042129': ['n59149006'], 'n15003831': ['n59149007'], 'n15010366': ['n59149008'], 'n15011452': ['n59149009'], 'n15013631': ['n59149010'], 'n15033661': ['n59149011'], 'n15034750': ['n59149012'], 'n15040195': ['n59149013'], 'n15034519': ['n59149014'], 'n15009381': ['n59149015'], 'n15021459': ['n59149016'], 'n15047846': ['n59149017'], 'n15046998': ['n59149018'], 'n15046519': ['n59149019'], 'n15026900': ['n59149020'], 'n15009375': ['n59149021'], 'n15030160': ['n59149022'], 'n15035850': ['n59149023'], 'n15011202': ['n59149024'], 'n15033670': ['n59149025'], 'n15031491': ['n59149026'], 'n15031016': ['n59149027'], 'n15033679': ['n59149028'], 'n15034526': ['n59149029'], 'n15032109': ['n59149030'], 'n15035854': ['n59149031'], 'n15013865': ['n59149032'], 'n15045892': ['n59149033'], 'n15018079': ['n59149034'], 'n15009359': ['n59149035'], 'n15044569': ['n59149036'], 'n15009116': ['n59149037'], 'n15000302': ['n59149038'], 'n15007185': ['n59149039'], 'n15035622': ['n59149040'], 'n15009368': ['n59149041'], 'n15031265': ['n59149042'], 'n15036718': ['n59149043'], 'n15014944': ['n59149044'], 'n15000377': ['n59149045'], 'n15022167': ['n59149046'], 'n15048316': ['n59149047'], 'n15001220': ['n59149048'], 'n15005828': ['n59149049'], 'n15027610': ['n59149050'], 'n15025673': ['n59149051'], 'n15011271': ['n59149052'], 'n15036570': ['n59149053'], 'n15026768': ['n59149054'], 'n15017803': ['n59149055'], 'n15034386': ['n59149056'], 'n15036323': ['n59149057'], 'n15037896': ['n59149058'], 'n15015864': ['n59149059'], 'n15016958': ['n59149060'], 'n15034145': ['n59149061'], 'n15038509': ['n59149062'], 'n15047471': ['n59149063'], 'n15046380': ['n59149064'], 'n15017802': ['n59149065'], 'n15003877': ['n59149066'], 'n15047211': ['n59149067'], 'n15025204': ['n59149068'], 'n15047451': ['n59149069'], 'n15003639': ['n59149070'], 'n15046368': ['n59149071'], 'n15024351': ['n59149072'], 'n15048304': ['n59149073'], 'n15047456': ['n59149074'], 'n15048787': ['n59149075'], 'n15004720': ['n59149076'], 'n15003875': ['n59149077'], 'n15002545': ['n59149078'], 'n15040834': ['n59149079'], 'n15011499': ['n59149080'], 'n15038996': ['n59149081'], 'n15016707': ['n59149082'], 'n15036572': ['n59149083'], 'n15016708': ['n59149084'], 'n15037663': ['n59149085'], 'n15026780': ['n59149086'], 'n15035008': ['n59149087'], 'n15015618': ['n59149088'], 'n15045282': ['n59149089'], 'n15010169': ['n59149090'], 'n15044191': ['n59149091'], 'n15013437': ['n59149092'], 'n15045064': ['n59149093'], 'n15027876': ['n59149094'], 'n15000352': ['n59149095'], 'n15025210': ['n59149096'], 'n15030807': ['n59149097'], 'n15025454': ['n59149098'], 'n15030808': ['n59149099'], 'n15002530': ['n59149100'], 'n15040605': ['n59149101'], 'n15040847': ['n59149102'], 'n15003622': ['n59149103'], 'n15001688': ['n59149104'], 'n15026549': ['n59149105'], 'n15012576': ['n59149106'], 'n15012577': ['n59149107'], 'n15035499': ['n59149108'], 'n15014992': ['n59149109'], 'n15035495': ['n59149110'], 'n15036101': ['n59149111'], 'n15039611': ['n59149112'], 'n15046162': ['n59149113'], 'n15026799': ['n59149114'], 'n15023288': ['n59149115'], 'n15049653': ['n59149116'], 'n15001674': ['n59149117'], 'n15025227': ['n59149118'], 'n15025462': ['n59149119'], 'n15047479': ['n59149120'], 'n15027882': ['n59149121'], 'n15023045': ['n59149122'], 'n15027641': ['n59149123'], 'n15004941': ['n59149124'], 'n15038777': ['n59149125'], 'n15033087': ['n59149126'], 'n15037204': ['n59149127'], 'n15010146': ['n59149128'], 'n15010145': ['n59149129'], 'n15045061': ['n59149130'], 'n15047421': ['n59149131'], 'n15048751': ['n59149132'], 'n15020183': ['n59149133'], 'n15020182': ['n59149134'], 'n15023691': ['n59149135'], 'n15024541': ['n59149136'], 'n15016760': ['n59149137'], 'n15002593': ['n59149138'], 'n15015671': ['n59149139'], 'n15013251': ['n59149140'], 'n15033493': ['n59149141'], 'n15033492': ['n59149142'], 'n15011063': ['n59149143'], 'n15031076': ['n59149144'], 'n15033497': ['n59149145'], 'n15034586': ['n59149146'], 'n15005857': ['n59149147'], 'n15044143': ['n59149148'], 'n15001255': ['n59149149'], 'n15006705': ['n59149150'], 'n15001012': ['n59149151'], 'n15044387': ['n59149152'], 'n15045475': ['n59149153'], 'n15001251': ['n59149154'], 'n15049839': ['n59149155'], 'n15047417': ['n59149156'], 'n15006942': ['n59149157'], 'n15025649': ['n59149158'], 'n15012146': ['n59149159'], 'n15038719': ['n59149160'], 'n15033269': ['n59149161'], 'n15045480': ['n59149162'], 'n15014329': ['n59149163'], 'n15021055': ['n59149164'], 'n15027833': ['n59149165'], 'n15046595': ['n59149166'], 'n15006938': ['n59149167'], 'n15025896': ['n59149168'], 'n15044178': ['n59149169'], 'n15046115': ['n59149170'], 'n15000393': ['n59149171'], 'n15027835': ['n59149172'], 'n15001006': ['n59149173'], 'n15012378': ['n59149174'], 'n15014799': ['n59149175'], 'n15012137': ['n59149176'], 'n15036303': ['n59149177'], 'n15018919': ['n59149178'], 'n15033277': ['n59149179'], 'n15013220': ['n59149180'], 'n15014319': ['n59149181'], 'n15017821': ['n59149182'], 'n15017822': ['n59149183'], 'n15013469': ['n59149184'], 'n15000389': ['n59149185'], 'n15004989': ['n59149186'], 'n15026997': ['n59149187'], 'n15026754': ['n59149188'], 'n15024572': ['n59149189'], 'n15043078': ['n59149190'], 'n15022394': ['n59149191'], 'n15047677': ['n59149192'], 'n15000141': ['n59149193'], 'n15046344': ['n59149194'], 'n15045256': ['n59149195'], 'n15046348': ['n59149196'], 'n15003653': ['n59149197'], 'n15004985': ['n59149198'], 'n15023248': ['n59149199'], 'n15025668': ['n59149200'], 'n15034381': ['n59149201'], 'n15033290': ['n59149202'], 'n15014304': ['n59149203'], 'n15032197': ['n59149204'], 'n15038977': ['n59149205'], 'n15010189': ['n59149206'], 'n15016728': ['n59149207'], 'n15039823': ['n59149208'], 'n15034134': ['n59149209'], 'n15023492': ['n59149210'], 'n15035469': ['n59149211'], 'n15038979': ['n59149212'], 'n15045261': ['n59149213'], 'n15049126': ['n59149214'], 'n15049368': ['n59149215'], 'n15009906': ['n59149216'], 'n15029999': ['n59149217'], 'n15027574': ['n59149218'], 'n15027333': ['n59149219'], 'n15015110': ['n59149220'], 'n15004213': ['n59149221'], 'n15039563': ['n59149222'], 'n15004698': ['n59149223'], 'n15044911': ['n59149224'], 'n15005306': ['n59149225'], 'n15009901': ['n59149226'], 'n15040305': ['n59149227'], 'n15038468': ['n59149228'], 'n15043826': ['n59149229'], 'n15013168': ['n59149230'], 'n15015588': ['n59149231'], 'n15040786': ['n59149232'], 'n15020709': ['n59149233'], 'n15039798': ['n59149234'], 'n15017769': ['n59149235'], 'n15030745': ['n59149236'], 'n15002029': ['n59149237'], 'n15031607': ['n59149238'], 'n15029525': ['n59149239'], 'n15000088': ['n59149240'], 'n15004683': ['n59149241'], 'n15004444': ['n59149242'], 'n15041873': ['n59149243'], 'n15037154': ['n59149244'], 'n15005777': ['n59149245'], 'n15040797': ['n59149246'], 'n15040796': ['n59149247'], 'n15004451': ['n59149248'], 'n15003120': ['n59149249'], 'n15039324': ['n59149250'], 'n15049360': ['n59149251'], 'n15025174': ['n59149252'], 'n15016425': ['n59149253'], 'n15017999': ['n59149254'], 'n15025171': ['n59149255'], 'n15026029': ['n59149256'], 'n15042513': ['n59149257'], 'n15043847': ['n59149258'], 'n15015569': ['n59149259'], 'n15041428': ['n59149260'], 'n15040326': ['n59149261'], 'n15041429': ['n59149262'], 'n15048061': ['n59149263'], 'n15016656': ['n59149264'], 'n15016658': ['n59149265'], 'n15030539': ['n59149266'], 'n15026038': ['n59149267'], 'n15029548': ['n59149268'], 'n15001396': ['n59149269'], 'n15029787': ['n59149270'], 'n15049138': ['n59149271'], 'n15042501': ['n59149272'], 'n15042745': ['n59149273'], 'n15015315': ['n59149274'], 'n15017738': ['n59149275'], 'n15044928': ['n59149276'], 'n15013375': ['n59149277'], 'n15014220': ['n59149278'], 'n15014462': ['n59149279'], 'n15000071': ['n59149280'], 'n15016884': ['n59149281'], 'n15032954': ['n59149282'], 'n15032955': ['n59149283'], 'n15025358': ['n59149284'], 'n15028621': ['n59149285'], 'n15049568': ['n59149286'], 'n15004496': ['n59149287'], 'n15009943': ['n59149288'], 'n15008852': ['n59149289'], 'n15040747': ['n59149290'], 'n15005595': ['n59149291'], 'n15014055': ['n59149292'], 'n15036003': ['n59149293'], 'n15038422': ['n59149294'], 'n15048483': ['n59149295'], 'n15022091': ['n59149296'], 'n15030943': ['n59149297'], 'n15028637': ['n59149298'], 'n15049797': ['n59149299'], 'n15018410': ['n59149300'], 'n15038683': ['n59149301'], 'n15036021': ['n59149302'], 'n15041832': ['n59149303'], 'n15040758': ['n59149304'], 'n15040756': ['n59149305'], 'n15037106': ['n59149306'], 'n15041837': ['n59149307'], 'n15006672': ['n59149308'], 'n15040510': ['n59149309'], 'n15003162': ['n59149310'], 'n15004253': ['n59149311'], 'n15007760': ['n59149312'], 'n15035168': ['n59149313'], 'n15046291': ['n59149314'], 'n15038439': ['n59149315'], 'n15027558': ['n59149316'], 'n15029978': ['n59149317'], 'n15029977': ['n59149318'], 'n15049586': ['n59149319'], 'n15016220': ['n59149320'], 'n15031818': ['n59149321'], 'n15014285': ['n59149322'], 'n15039780': ['n59149323'], 'n15015371': ['n59149324'], 'n15004237': ['n59149325'], 'n15013192': ['n59149326'], 'n15035184': ['n59149327'], 'n15034093': ['n59149328'], 'n15019728': ['n59149329'], 'n15014038': ['n59149330'], 'n15039532': ['n59149331'], 'n15002063': ['n59149332'], 'n15005330': ['n59149333'], 'n15028893': ['n59149334'], 'n15016455': ['n59149335'], 'n15030722': ['n59149336'], 'n15030721': ['n59149337'], 'n15016218': ['n59149338'], 'n15025147': ['n59149339'], 'n15029746': ['n59149340'], 'n15009916': ['n59149341'], 'n15029745': ['n59149342'], 'n15029742': ['n59149343'], 'n15006641': ['n59149344'], 'n15037373': ['n59149345'], 'n15006403': ['n59149346'], 'n15007976': ['n59149347'], 'n15039300': ['n59149348'], 'n15038456': ['n59149349'], 'n15002053': ['n59149350'], 'n15005561': ['n59149351'], 'n15039544': ['n59149352'], 'n15024061': ['n59149353'], 'n15025152': ['n59149354'], 'n15047163': ['n59149355'], 'n15016205': ['n59149356'], 'n15030732': ['n59149357'], 'n15030977': ['n59149358'], 'n15049340': ['n59149359'], 'n15019792': ['n59149360'], 'n15020783': ['n59149361'], 'n15029358': ['n59149362'], 'n15005145': ['n59149363'], 'n15023813': ['n59149364'], 'n15006476': ['n59149365'], 'n15009986': ['n59149366'], 'n15020548': ['n59149367'], 'n15044515': ['n59149368'], 'n15020547': ['n59149369'], 'n15021875': ['n59149370'], 'n15023812': ['n59149371'], 'n15046935': ['n59149372'], 'n15008662': ['n59149373'], 'n15000937': ['n59149374'], 'n15007331': ['n59149375'], 'n15015188': ['n59149376'], 'n15032766': ['n59149377'], 'n15011823': ['n59149378'], 'n15018458': ['n59149379'], 'n15012909': ['n59149380'], 'n15016271': ['n59149381'], 'n15034719': ['n59149382'], 'n15045830': ['n59149383'], 'n15018212': ['n59149384'], 'n15028037': ['n59149385'], 'n15016276': ['n59149386'], 'n15043893': ['n59149387'], 'n15045833': ['n59149388'], 'n15045831': ['n59149389'], 'n15014094': ['n59149390'], 'n15015183': ['n59149391'], 'n15004290': ['n59149392'], 'n15004053': ['n59149393'], 'n15009984': ['n59149394'], 'n15031441': ['n59149395'], 'n15004055': ['n59149396'], 'n15030595': ['n59149397'], 'n15030110': ['n59149398'], 'n15018687': ['n59149399'], 'n15019533': ['n59149400'], 'n15032535': ['n59149401'], 'n15018447': ['n59149402'], 'n15011813': ['n59149403'], 'n15031202': ['n59149404'], 'n15022981': ['n59149405'], 'n15022982': ['n59149406'], 'n15038092': ['n59149407'], 'n15034729': ['n59149408'], 'n15020563': ['n59149409'], 'n15046951': ['n59149410'], 'n15042594': ['n59149411'], 'n15043440': ['n59149412'], 'n15024925': ['n59149413'], 'n15009965': ['n59149414'], 'n15047807': ['n59149415'], 'n15023831': ['n59149416'], 'n15046716': ['n59149417'], 'n15031691': ['n59149418'], 'n15014079': ['n59149419'], 'n15014078': ['n59149420'], 'n15032781': ['n59149421'], 'n15032782': ['n59149422'], 'n15028057': ['n59149423'], 'n15031454': ['n59149424'], 'n15031455': ['n59149425'], 'n15030128': ['n59149426'], 'n15019528': ['n59149427'], 'n15004266': ['n59149428'], 'n15020336': ['n59149429'], 'n15042102': ['n59149430'], 'n15007774': ['n59149431'], 'n15008864': ['n59149432'], 'n15041257': ['n59149433'], 'n15023606': ['n59149434'], 'n15006207': ['n59149435'], 'n15014070': ['n59149436'], 'n15023844': ['n59149437'], 'n15002094': ['n59149438'], 'n15007542': ['n59149439'], 'n15003185': ['n59149440'], 'n15030374': ['n59149441'], 'n15034975': ['n59149442'], 'n15017336': ['n59149443'], 'n15033408': ['n59149444'], 'n15034736': ['n59149445'], 'n15029559': ['n59149446'], 'n15019112': ['n59149447'], 'n15016089': ['n59149448'], 'n15041442': ['n59149449'], 'n15041685': ['n59149450'], 'n15007367': ['n59149451'], 'n15038033': ['n59149452'], 'n15042777': ['n59149453'], 'n15037183': ['n59149454'], 'n15039359': ['n59149455'], 'n15009791': ['n59149456'], 'n15039114': ['n59149457'], 'n15006285': ['n59149458'], 'n15038269': ['n59149459'], 'n15006044': ['n59149460'], 'n15018499': ['n59149461'], 'n15018017': ['n59149462'], 'n15029561': ['n59149463'], 'n15026292': ['n59149464'], 'n15030787': ['n59149465'], 'n15030789': ['n59149466'], 'n15019581': ['n59149467'], 'n15019580': ['n59149468'], 'n15002900': ['n59149469'], 'n15028234': ['n59149470'], 'n15006269': ['n59149471'], 'n15022933': ['n59149472'], 'n15022936': ['n59149473'], 'n15044702': ['n59149474'], 'n15032970': ['n59149475'], 'n15038038': ['n59149476'], 'n15039368': ['n59149477'], 'n15018247': ['n59149478'], 'n15031643': ['n59149479'], 'n15027392': ['n59149480'], 'n15010521': ['n59149481'], 'n15012944': ['n59149482'], 'n15031406': ['n59149483'], 'n15019570': ['n59149484'], 'n15032507': ['n59149485'], 'n15020760': ['n59149486'], 'n15044732': ['n59149487'], 'n15006017': ['n59149488'], 'n15006256': ['n59149489'], 'n15038056': ['n59149490'], 'n15039145': ['n59149491'], 'n15039387': ['n59149492'], 'n15006499': ['n59149493'], 'n15008434': ['n59149494'], 'n15042317': ['n59149495'], 'n15005175': ['n59149496'], 'n15007354': ['n59149497'], 'n15008443': ['n59149498'], 'n15039135': ['n59149499'], 'n15004088': ['n59149500'], 'n15008442': ['n59149501'], 'n15029344': ['n59149502'], 'n15032501': ['n59149503'], 'n15031894': ['n59149504'], 'n15032983': ['n59149505'], 'n15040361': ['n59149506'], 'n15010996': ['n59149507'], 'n15049196': ['n59149508'], 'n15016051': ['n59149509'], 'n15049179': ['n59149510'], 'n15049178': ['n59149511'], 'n15020530': ['n59149512'], 'n15028016': ['n59149513'], 'n15018476': ['n59149514'], 'n15041451': ['n59149515'], 'n15006487': ['n59149516'], 'n15042303': ['n59149517'], 'n15028024': ['n59149518'], 'n15028023': ['n59149519'], 'n15031422': ['n59149520'], 'n15027170': ['n59149521'], 'n15045560': ['n59149522'], 'n15046412': ['n59149523'], 'n15047986': ['n59149524'], 'n15043388': ['n59149525'], 'n15043387': ['n59149526'], 'n15046895': ['n59149527'], 'n15009227': ['n59149528'], 'n15024866': ['n59149529'], 'n15026803': ['n59149530'], 'n15025957': ['n59149531'], 'n15032240': ['n59149532'], 'n15009231': ['n59149533'], 'n15036600': ['n59149534'], 'n15011783': ['n59149535'], 'n15033334': ['n59149536'], 'n15009476': ['n59149537'], 'n15001509': ['n59149538'], 'n15034420': ['n59149539'], 'n15035518': ['n59149540'], 'n15009470': ['n59149541'], 'n15010213': ['n59149542'], 'n15023784': ['n59149543'], 'n15023783': ['n59149544'], 'n15018175': ['n59149545'], 'n15018178': ['n59149546'], 'n15041199': ['n59149547'], 'n15009458': ['n59149548'], 'n15045314': ['n59149549'], 'n15000646': ['n59149550'], 'n15023303': ['n59149551'], 'n15001972': ['n59149552'], 'n15000643': ['n59149553'], 'n15011531': ['n59149554'], 'n15029093': ['n59149555'], 'n15034670': ['n59149556'], 'n15006194': ['n59149557'], 'n15032255': ['n59149558'], 'n15031167': ['n59149559'], 'n15034674': ['n59149560'], 'n15042290': ['n59149561'], 'n15023794': ['n59149562'], 'n15000871': ['n59149563'], 'n15019498': ['n59149564'], 'n15048610': ['n59149565'], 'n15044016': ['n59149566'], 'n15044019': ['n59149567'], 'n15045347': ['n59149568'], 'n15047767': ['n59149569'], 'n15047529': ['n59149570'], 'n15035534': ['n59149571'], 'n15011525': ['n59149572'], 'n15010435': ['n59149573'], 'n15043170': ['n59149574'], 'n15019481': ['n59149575'], 'n15042062': ['n59149576'], 'n15017063': ['n59149577'], 'n15023566': ['n59149578'], 'n15025744': ['n59149579'], 'n15049932': ['n59149580'], 'n15022477': ['n59149581'], 'n15027920': ['n59149582'], 'n15022470': ['n59149583'], 'n15024891': ['n59149584'], 'n15044243': ['n59149585'], 'n15025749': ['n59149586'], 'n15035790': ['n59149587'], 'n15023328': ['n59149588'], 'n15031190': ['n59149589'], 'n15032273': ['n59149590'], 'n15034452': ['n59149591'], 'n15039901': ['n59149592'], 'n15011517': ['n59149593'], 'n15012848': ['n59149594'], 'n15033369': ['n59149595'], 'n15022881': ['n59149596'], 'n15000696': ['n59149597'], 'n15002878': ['n59149598'], 'n15003726': ['n59149599'], 'n15003969': ['n59149600'], 'n15044673': ['n59149601'], 'n15022880': ['n59149602'], 'n15046614': ['n59149603'], 'n15042017': ['n59149604'], 'n15023732': ['n59149605'], 'n15021556': ['n59149606'], 'n15020227': ['n59149607'], 'n15031591': ['n59149608'], 'n15013762': ['n59149609'], 'n15032443': ['n59149610'], 'n15009277': ['n59149611'], 'n15025919': ['n59149612'], 'n15031113': ['n59149613'], 'n15033775': ['n59149614'], 'n15034864': ['n59149615'], 'n15013527': ['n59149616'], 'n15011588': ['n59149617'], 'n15033538': ['n59149618'], 'n15003713': ['n59149619'], 'n15024830': ['n59149620'], 'n15020233': ['n59149621'], 'n15041157': ['n59149622'], 'n15047933': ['n59149623'], 'n15001538': ['n59149624'], 'n15047932': ['n59149625'], 'n15025923': ['n59149626'], 'n15002621': ['n59149627'], 'n15025922': ['n59149628'], 'n15011576': ['n59149629'], 'n15031120': ['n59149630'], 'n15032694': ['n59149631'], 'n15033782': ['n59149632'], 'n15013759': ['n59149633'], 'n15029054': ['n59149634'], 'n15001522': ['n59149635'], 'n15001764': ['n59149636'], 'n15002612': ['n59149637'], 'n15024842': ['n59149638'], 'n15002857': ['n59149639'], 'n15021575': ['n59149640'], 'n15045302': ['n59149641'], 'n15020484': ['n59149642'], 'n15022660': ['n59149643'], 'n15021334': ['n59149644'], 'n15002853': ['n59149645'], 'n15013742': ['n59149646'], 'n15009251': ['n59149647'], 'n15033551': ['n59149648'], 'n15013508': ['n59149649'], 'n15015927': ['n59149650'], 'n15034402': ['n59149651'], 'n15012657': ['n59149652'], 'n15029065': ['n59149653'], 'n15011569': ['n59149654'], 'n15011325': ['n59149655'], 'n15034648': ['n59149656'], 'n15035978': ['n59149657'], 'n15022431': ['n59149658'], 'n15019046': ['n59149659'], 'n15046622': ['n59149660'], 'n15002608': ['n59149661'], 'n15019047': ['n59149662'], 'n15042269': ['n59149663'], 'n15043117': ['n59149664'], 'n15046866': ['n59149665'], 'n15000426': ['n59149666'], 'n15048805': ['n59149667'], 'n15030293': ['n59149668'], 'n15034895': ['n59149669'], 'n15034894': ['n59149670'], 'n15031147': ['n59149671'], 'n15032478': ['n59149672'], 'n15012888': ['n59149673'], 'n15035744': ['n59149674'], 'n15011558': ['n59149675'], 'n15012403': ['n59149676'], 'n15000015': ['n59149677'], 'n15021198': ['n59149678'], 'n15047344': ['n59149679'], 'n15026404': ['n59149680'], 'n15029912': ['n59149681'], 'n15046497': ['n59149682'], 'n15025314': ['n59149683'], 'n15005708': ['n59149684'], 'n15025553': ['n59149685'], 'n15046256': ['n59149686'], 'n15000250': ['n59149687'], 'n15005940': ['n59149688'], 'n15048438': ['n59149689'], 'n15026408': ['n59149690'], 'n15027736': ['n59149691'], 'n15004612': ['n59149692'], 'n15027737': ['n59149693'], 'n15010057': ['n59149694'], 'n15012231': ['n59149695'], 'n15035116': ['n59149696'], 'n15015740': ['n59149697'], 'n15039957': ['n59149698'], 'n15049994': ['n59149699'], 'n15010290': ['n59149700'], 'n15049759': ['n59149701'], 'n15010291': ['n59149702'], 'n15048427': ['n59149703'], 'n15013797': ['n59149704'], 'n15013798': ['n59149705'], 'n15039964': ['n59149706'], 'n15009065': ['n59149707'], 'n15033184': ['n59149708'], 'n15037789': ['n59149709'], 'n15013552': ['n59149710'], 'n15037784': ['n59149711'], 'n15001580': ['n59149712'], 'n15044071': ['n59149713'], 'n15046492': ['n59149714'], 'n15044097': ['n59149715'], 'n15024003': ['n59149716'], 'n15024487': ['n59149717'], 'n15027999': ['n59149718'], 'n15005925': ['n59149719'], 'n15026666': ['n59149720'], 'n15027510': ['n59149721'], 'n15027995': ['n59149722'], 'n15024243': ['n59149723'], 'n15049545': ['n59149724'], 'n15049309': ['n59149725'], 'n15035382': ['n59149726'], 'n15027759': ['n59149727'], 'n15015722': ['n59149728'], 'n15034285': ['n59149729'], 'n15037315': ['n59149730'], 'n15040721': ['n59149731'], 'n15009053': ['n59149732'], 'n15016816': ['n59149733'], 'n15038888': ['n59149734'], 'n15039977': ['n59149735'], 'n15012692': ['n59149736'], 'n15023161': ['n59149737'], 'n15047372': ['n59149738'], 'n15015967': ['n59149739'], 'n15037319': ['n59149740'], 'n15039739': ['n59149741'], 'n15001797': ['n59149742'], 'n15029704': ['n59149743'], 'n15029946': ['n59149744'], 'n15027763': ['n59149745'], 'n15047356': ['n59149746'], 'n15002882': ['n59149747'], 'n15027529': ['n59149748'], 'n15011112': ['n59149749'], 'n15035388': ['n59149750'], 'n15014622': ['n59149751'], 'n15036478': ['n59149752'], 'n15014621': ['n59149753'], 'n15038899': ['n59149754'], 'n15015950': ['n59149755'], 'n15010269': ['n59149756'], 'n15039506': ['n59149757'], 'n15006839': ['n59149758'], 'n15026841': ['n59149759'], 'n15046692': ['n59149760'], 'n15025512': ['n59149761'], 'n15045362': ['n59149762'], 'n15049966': ['n59149763'], 'n15020068': ['n59149764'], 'n15025993': ['n59149765'], 'n15001383': ['n59149766'], 'n15047301': ['n59149767'], 'n15003568': ['n59149768'], 'n15016879': ['n59149769'], 'n15012035': ['n59149770'], 'n15018818': ['n59149771'], 'n15032282': ['n59149772'], 'n15032283': ['n59149773'], 'n15013365': ['n59149774'], 'n15032289': ['n59149775'], 'n15001151': ['n59149776'], 'n15033137': ['n59149777'], 'n15036889': ['n59149778'], 'n15038826': ['n59149779'], 'n15023101': ['n59149780'], 'n15024675': ['n59149781'], 'n15000047': ['n59149782'], 'n15002228': ['n59149783'], 'n15027944': ['n59149784'], 'n15042085': ['n59149785'], 'n15026611': ['n59149786'], 'n15045598': ['n59149787'], 'n15049712': ['n59149788'], 'n15023583': ['n59149789'], 'n15014691': ['n59149790'], 'n15002464': ['n59149791'], 'n15048867': ['n59149792'], 'n15014204': ['n59149793'], 'n15014446': ['n59149794'], 'n15012024': ['n59149795'], 'n15034230': ['n59149796'], 'n15002471': ['n59149797'], 'n15016862': ['n59149798'], 'n15016622': ['n59149799'], 'n15035326': ['n59149800'], 'n15047780': ['n59149801'], 'n15036658': ['n59149802'], 'n15037988': ['n59149803'], 'n15045385': ['n59149804'], 'n15022024': ['n59149805'], 'n15006815': ['n59149806'], 'n15049745': ['n59149807'], 'n15049984': ['n59149808'], 'n15022269': ['n59149809'], 'n15023116': ['n59149810'], 'n15002455': ['n59149811'], 'n15023358': ['n59149812'], 'n15028803': ['n59149813'], 'n15033154': ['n59149814'], 'n15037512': ['n59149815'], 'n15012498': ['n59149816'], 'n15034482': ['n59149817'], 'n15003790': ['n59149818'], 'n15010075': ['n59149819'], 'n15033155': ['n59149820'], 'n15035332': ['n59149821'], 'n15009096': ['n59149822'], 'n15002689': ['n59149823'], 'n15004628': ['n59149824'], 'n15044044': ['n59149825'], 'n15001112': ['n59149826'], 'n15000261': ['n59149827'], 'n15047796': ['n59149828'], 'n15048401': ['n59149829'], 'n15002685': ['n59149830'], 'n15005953': ['n59149831'], 'n15014424': ['n59149832'], 'n15037765': ['n59149833'], 'n15040931': ['n59149834'], 'n15010068': ['n59149835'], 'n15034251': ['n59149836'], 'n15033168': ['n59149837'], 'n15037521': ['n59149838'], 'n15035109': ['n59149839'], 'n15097323': ['n59149840'], 'n15096233': ['n59149841'], 'n15094057': ['n59149842'], 'n15097566': ['n59149843'], 'n15076862': ['n59149844'], 'n15077952': ['n59149845'], 'n15052654': ['n59149846'], 'n15097329': ['n59149847'], 'n15076623': ['n59149848'], 'n15077953': ['n59149849'], 'n15070081': ['n59149850'], 'n15070082': ['n59149851'], 'n15066817': ['n59149852'], 'n15070083': ['n59149853'], 'n15086665': ['n59149854'], 'n15090934': ['n59149855'], 'n15085577': ['n59149856'], 'n15062216': ['n59149857'], 'n15062210': ['n59149858'], 'n15063780': ['n59149859'], 'n15061120': ['n59149860'], 'n15073119': ['n59149861'], 'n15052659': ['n59149862'], 'n15076636': ['n59149863'], 'n15094288': ['n59149864'], 'n15053732': ['n59149865'], 'n15073368': ['n59149866'], 'n15097790': ['n59149867'], 'n15074210': ['n59149868'], 'n15065958': ['n59149869'], 'n15071183': ['n59149870'], 'n15061594': ['n59149871'], 'n15088858': ['n59149872'], 'n15060026': ['n59149873'], 'n15088615': ['n59149874'], 'n15075515': ['n59149875'], 'n15099720': ['n59149876'], 'n15050253': ['n59149877'], 'n15072248': ['n59149878'], 'n15074426': ['n59149879'], 'n15053521': ['n59149880'], 'n15073336': ['n59149881'], 'n15075750': ['n59149882'], 'n15086408': ['n59149883'], 'n15086643': ['n59149884'], 'n15063325': ['n59149885'], 'n15055941': ['n59149886'], 'n15055700': ['n59149887'], 'n15062230': ['n59149888'], 'n15077938': ['n59149889'], 'n15076849': ['n59149890'], 'n15097774': ['n59149891'], 'n15098864': ['n59149892'], 'n15069089': ['n59149893'], 'n15099951': ['n59149894'], 'n15052425': ['n59149895'], 'n15074437': ['n59149896'], 'n15050244': ['n59149897'], 'n15051572': ['n59149898'], 'n15072010': ['n59149899'], 'n15064409': ['n59149900'], 'n15086897': ['n59149901'], 'n15090901': ['n59149902'], 'n15065737': ['n59149903'], 'n15085326': ['n59149904'], 'n15087503': ['n59149905'], 'n15086651': ['n59149906'], 'n15095585': ['n59149907'], 'n15094013': ['n59149908'], 'n15052610': ['n59149909'], 'n15059252': ['n59149910'], 'n15060235': ['n59149911'], 'n15058159': ['n59149912'], 'n15059249': ['n59149913'], 'n15060233': ['n59149914'], 'n15081170': ['n59149915'], 'n15053948': ['n59149916'], 'n15068197': ['n59149917'], 'n15083592': ['n59149918'], 'n15053934': ['n59149919'], 'n15073323': ['n59149920'], 'n15077923': ['n59149921'], 'n15097996': ['n59149922'], 'n15095336': ['n59149923'], 'n15094006': ['n59149924'], 'n15093398': ['n59149925'], 'n15064828': ['n59149926'], 'n15093154': ['n59149927'], 'n15094482': ['n59149928'], 'n15096661': ['n59149929'], 'n15062886': ['n59149930'], 'n15063735': ['n59149931'], 'n15085303': ['n59149932'], 'n15069070': ['n59149933'], 'n15053939': ['n59149934'], 'n15053723': ['n59149935'], 'n15072441': ['n59149936'], 'n15076802': ['n59149937'], 'n15053964': ['n59149938'], 'n15072682': ['n59149939'], 'n15076803': ['n59149940'], 'n15073776': ['n59149941'], 'n15050453': ['n59149942'], 'n15085758': ['n59149943'], 'n15086848': ['n59149944'], 'n15057097': ['n59149945'], 'n15094471': ['n59149946'], 'n15085752': ['n59149947'], 'n15058176': ['n59149948'], 'n15080065': ['n59149949'], 'n15064851': ['n59149950'], 'n15063521': ['n59149951'], 'n15097507': ['n59149952'], 'n15097506': ['n59149953'], 'n15062670': ['n59149954'], 'n15070028': ['n59149955'], 'n15072207': ['n59149956'], 'n15074627': ['n59149957'], 'n15050218': ['n59149958'], 'n15051548': ['n59149959'], 'n15055902': ['n59149960'], 'n15096883': ['n59149961'], 'n15075965': ['n59149962'], 'n15092040': ['n59149963'], 'n15052623': ['n59149964'], 'n15069047': ['n59149965'], 'n15053951': ['n59149966'], 'n15086618': ['n59149967'], 'n15082018': ['n59149968'], 'n15059022': ['n59149969'], 'n15058166': ['n59149970'], 'n15059499': ['n59149971'], 'n15060489': ['n59149972'], 'n15083343': ['n59149973'], 'n15085525': ['n59149974'], 'n15062420': ['n59149975'], 'n15061575': ['n59149976'], 'n15076819': ['n59149977'], 'n15069054': ['n59149978'], 'n15077906': ['n59149979'], 'n15054439': ['n59149980'], 'n15073199': ['n59149981'], 'n15053105': ['n59149982'], 'n15050077': ['n59149983'], 'n15085180': ['n59149984'], 'n15075137': ['n59149985'], 'n15054432': ['n59149986'], 'n15079732': ['n59149987'], 'n15077312': ['n59149988'], 'n15077796': ['n59149989'], 'n15077311': ['n59149990'], 'n15098496': ['n59149991'], 'n15080968': ['n59149992'], 'n15064238': ['n59149993'], 'n15067749': ['n59149994'], 'n15091868': ['n59149995'], 'n15067746': ['n59149996'], 'n15064234': ['n59149997'], 'n15065326': ['n59149998'], 'n15090775': ['n59149999'], 'n15062294': ['n59150000'], 'n15085173': ['n59150001'], 'n15086262': ['n59150002'], 'n15088202': ['n59150003'], 'n15052008': ['n59150004'], 'n15099579': ['n59150005'], 'n15099578': ['n59150006'], 'n15050063': ['n59150007'], 'n15082910': ['n59150008'], 'n15090761': ['n59150009'], 'n15078651': ['n59150010'], 'n15079743': ['n59150011'], 'n15076233': ['n59150012'], 'n15097154': ['n59150013'], 'n15077560': ['n59150014'], 'n15063138': ['n59150015'], 'n15092705': ['n59150016'], 'n15089309': ['n59150017'], 'n15092702': ['n59150018'], 'n15056840': ['n59150019'], 'n15062289': ['n59150020'], 'n15066644': ['n59150021'], 'n15088697': ['n59150022'], 'n15086035': ['n59150023'], 'n15092709': ['n59150024'], 'n15078627': ['n59150025'], 'n15054698': ['n59150026'], 'n15079954': ['n59150027'], 'n15080941': ['n59150028'], 'n15090751': ['n59150029'], 'n15076201': ['n59150030'], 'n15097142': ['n59150031'], 'n15098231': ['n59150032'], 'n15051190': ['n59150033'], 'n15056870': ['n59150034'], 'n15063168': ['n59150035'], 'n15092933': ['n59150036'], 'n15058810': ['n59150037'], 'n15086249': ['n59150038'], 'n15087338': ['n59150039'], 'n15055303': ['n59150040'], 'n15055304': ['n59150041'], 'n15056877': ['n59150042'], 'n15055305': ['n59150043'], 'n15064491': ['n59150044'], 'n15057719': ['n59150045'], 'n15076455': ['n59150046'], 'n15053114': ['n59150047'], 'n15077788': ['n59150048'], 'n15078635': ['n59150049'], 'n15052264': ['n59150050'], 'n15076694': ['n59150051'], 'n15077540': ['n59150052'], 'n15054682': ['n59150053'], 'n15075123': ['n59150054'], 'n15096043': ['n59150055'], 'n15087106': ['n59150056'], 'n15089528': ['n59150057'], 'n15065338': ['n59150058'], 'n15090987': ['n59150059'], 'n15056861': ['n59150060'], 'n15056862': ['n59150061'], 'n15065334': ['n59150062'], 'n15089526': ['n59150063'], 'n15090984': ['n59150064'], 'n15064242': ['n59150065'], 'n15087580': ['n59150066'], 'n15062066': ['n59150067'], 'n15067991': ['n59150068'], 'n15050038': ['n59150069'], 'n15050039': ['n59150070'], 'n15072067': ['n59150071'], 'n15078606': ['n59150072'], 'n15051366': ['n59150073'], 'n15052212': ['n59150074'], 'n15079930': ['n59150075'], 'n15096272': ['n59150076'], 'n15050031': ['n59150077'], 'n15053780': ['n59150078'], 'n15075332': ['n59150079'], 'n15059096': ['n59150080'], 'n15074242': ['n59150081'], 'n15096032': ['n59150082'], 'n15065529': ['n59150083'], 'n15073152': ['n59150084'], 'n15074483': ['n59150085'], 'n15080924': ['n59150086'], 'n15073151': ['n59150087'], 'n15097364': ['n59150088'], 'n15091825': ['n59150089'], 'n15063108': ['n59150090'], 'n15089732': ['n59150091'], 'n15092911': ['n59150092'], 'n15089733': ['n59150093'], 'n15089734': ['n59150094'], 'n15087316': ['n59150095'], 'n15062250': ['n59150096'], 'n15064670': ['n59150097'], 'n15083192': ['n59150098'], 'n15057904': ['n59150099'], 'n15060072': ['n59150100'], 'n15078858': ['n59150101'], 'n15078857': ['n59150102'], 'n15051357': ['n59150103'], 'n15098689': ['n59150104'], 'n15099533': ['n59150105'], 'n15098688': ['n59150106'], 'n15054865': ['n59150107'], 'n15099539': ['n59150108'], 'n15051353': ['n59150109'], 'n15074017': ['n59150110'], 'n15074250': ['n59150111'], 'n15074011': ['n59150112'], 'n15066848': ['n59150113'], 'n15084059': ['n59150114'], 'n15060060': ['n59150115'], 'n15085381': ['n59150116'], 'n15088650': ['n59150117'], 'n15053328': ['n59150118'], 'n15076889': ['n59150119'], 'n15079913': ['n59150120'], 'n15096015': ['n59150121'], 'n15072286': ['n59150122'], 'n15074229': ['n59150123'], 'n15072048': ['n59150124'], 'n15052474': ['n59150125'], 'n15080900': ['n59150126'], 'n15096010': ['n59150127'], 'n15096493': ['n59150128'], 'n15065309': ['n59150129'], 'n15074461': ['n59150130'], 'n15062038': ['n59150131'], 'n15090954': ['n59150132'], 'n15067967': ['n59150133'], 'n15055980': ['n59150134'], 'n15087539': ['n59150135'], 'n15061184': ['n59150136'], 'n15055747': ['n59150137'], 'n15076415': ['n59150138'], 'n15079924': ['n59150139'], 'n15051135': ['n59150140'], 'n15077501': ['n59150141'], 'n15095159': ['n59150142'], 'n15052225': ['n59150143'], 'n15077986': ['n59150144'], 'n15096009': ['n59150145'], 'n15050041': ['n59150146'], 'n15076651': ['n59150147'], 'n15050284': ['n59150148'], 'n15080913': ['n59150149'], 'n15072052': ['n59150150'], 'n15064447': ['n59150151'], 'n15060088': ['n59150152'], 'n15060087': ['n59150153'], 'n15084039': ['n59150154'], 'n15090941': ['n59150155'], 'n15063599': ['n59150156'], 'n15068800': ['n59150157'], 'n15063356': ['n59150158'], 'n15089962': ['n59150159'], 'n15055736': ['n59150160'], 'n15056825': ['n59150161'], 'n15090947': ['n59150162'], 'n15066380': ['n59150163'], 'n15070880': ['n59150164'], 'n15070881': ['n59150165'], 'n15071731': ['n59150166'], 'n15069897': ['n59150167'], 'n15094853': ['n59150168'], 'n15071730': ['n59150169'], 'n15081534': ['n59150170'], 'n15054155': ['n59150171'], 'n15078125': ['n59150172'], 'n15061921': ['n59150173'], 'n15076183': ['n59150174'], 'n15054393': ['n59150175'], 'n15055485': ['n59150176'], 'n15091346': ['n59150177'], 'n15091347': ['n59150178'], 'n15087075': ['n59150179'], 'n15071737': ['n59150180'], 'n15094840': ['n59150181'], 'n15070651': ['n59150182'], 'n15070652': ['n59150183'], 'n15079228': ['n59150184'], 'n15090000': ['n59150185'], 'n15085907': ['n59150186'], 'n15093992': ['n59150187'], 'n15077290': ['n59150188'], 'n15090008': ['n59150189'], 'n15068306': ['n59150190'], 'n15080454': ['n59150191'], 'n15083963': ['n59150192'], 'n15067695': ['n59150193'], 'n15071506': ['n59150194'], 'n15089481': ['n59150195'], 'n15071952': ['n59150196'], 'n15078108': ['n59150197'], 'n15088390': ['n59150198'], 'n15092896': ['n59150199'], 'n15078101': ['n59150200'], 'n15075078': ['n59150201'], 'n15083939': ['n59150202'], 'n15076170': ['n59150203'], 'n15081990': ['n59150204'], 'n15092899': ['n59150205'], 'n15059864': ['n59150206'], 'n15068578': ['n59150207'], 'n15088385': ['n59150208'], 'n15087053': ['n59150209'], 'n15056118': ['n59150210'], 'n15067244': ['n59150211'], 'n15089471': ['n59150212'], 'n15069640': ['n59150213'], 'n15071961': ['n59150214'], 'n15056591': ['n59150215'], 'n15077021': ['n59150216'], 'n15057671': ['n59150217'], 'n15060840': ['n59150218'], 'n15057675': ['n59150219'], 'n15059855': ['n59150220'], 'n15066148': ['n59150221'], 'n15098193': ['n59150222'], 'n15098192': ['n59150223'], 'n15050806': ['n59150224'], 'n15094822': ['n59150225'], 'n15056106': ['n59150226'], 'n15057678': ['n59150227'], 'n15071725': ['n59150228'], 'n15059618': ['n59150229'], 'n15070840': ['n59150230'], 'n15088370': ['n59150231'], 'n15053027': ['n59150232'], 'n15078563': ['n59150233'], 'n15091301': ['n59150234'], 'n15069616': ['n59150235'], 'n15078323': ['n59150236'], 'n15098177': ['n59150237'], 'n15052173': ['n59150238'], 'n15088129': ['n59150239'], 'n15091308': ['n59150240'], 'n15066339': ['n59150241'], 'n15080884': ['n59150242'], 'n15088120': ['n59150243'], 'n15069866': ['n59150244'], 'n15082837': ['n59150245'], 'n15051073': ['n59150246'], 'n15082836': ['n59150247'], 'n15051080': ['n59150248'], 'n15090448': ['n59150249'], 'n15068506': ['n59150250'], 'n15081981': ['n59150251'], 'n15068986': ['n59150252'], 'n15066562': ['n59150253'], 'n15088130': ['n59150254'], 'n15089224': ['n59150255'], 'n15066563': ['n59150256'], 'n15053045': ['n59150257'], 'n15053287': ['n59150258'], 'n15060815': ['n59150259'], 'n15080623': ['n59150260'], 'n15075270': ['n59150261'], 'n15054371': ['n59150262'], 'n15070828': ['n59150263'], 'n15090435': ['n59150264'], 'n15090677': ['n59150265'], 'n15080861': ['n59150266'], 'n15066355': ['n59150267'], 'n15066595': ['n59150268'], 'n15056319': ['n59150269'], 'n15068770': ['n59150270'], 'n15088101': ['n59150271'], 'n15088584': ['n59150272'], 'n15089430': ['n59150273'], 'n15052187': ['n59150274'], 'n15075285': ['n59150275'], 'n15075040': ['n59150276'], 'n15097299': ['n59150277'], 'n15057871': ['n59150278'], 'n15065017': ['n59150279'], 'n15058961': ['n59150280'], 'n15065496': ['n59150281'], 'n15064161': ['n59150282'], 'n15056308': ['n59150283'], 'n15087266': ['n59150284'], 'n15067670': ['n59150285'], 'n15087265': ['n59150286'], 'n15092272': ['n59150287'], 'n15070482': ['n59150288'], 'n15095542': ['n59150289'], 'n15097722': ['n59150290'], 'n15083558': ['n59150291'], 'n15082462': ['n59150292'], 'n15079064': ['n59150293'], 'n15082221': ['n59150294'], 'n15083555': ['n59150295'], 'n15063940': ['n59150296'], 'n15061762': ['n59150297'], 'n15087910': ['n59150298'], 'n15095549': ['n59150299'], 'n15071337': ['n59150300'], 'n15072669': ['n59150301'], 'n15069268': ['n59150302'], 'n15092266': ['n59150303'], 'n15058350': ['n59150304'], 'n15091174': ['n59150305'], 'n15093593': ['n59150306'], 'n15080294': ['n59150307'], 'n15060423': ['n59150308'], 'n15079072': ['n59150309'], 'n15057258': ['n59150310'], 'n15094207': ['n59150311'], 'n15059438': ['n59150312'], 'n15072679': ['n59150313'], 'n15067053': ['n59150314'], 'n15052833': ['n59150315'], 'n15074820': ['n59150316'], 'n15074821': ['n59150317'], 'n15052832': ['n59150318'], 'n15096855': ['n59150319'], 'n15050890': ['n59150320'], 'n15064818': ['n59150321'], 'n15091167': ['n59150322'], 'n15061305': ['n59150323'], 'n15058141': ['n59150324'], 'n15084861': ['n59150325'], 'n15062874': ['n59150326'], 'n15083770': ['n59150327'], 'n15058379': ['n59150328'], 'n15059467': ['n59150329'], 'n15060210': ['n59150330'], 'n15078190': ['n59150331'], 'n15071559': ['n59150332'], 'n15094679': ['n59150333'], 'n15051508': ['n59150334'], 'n15060690': ['n59150335'], 'n15069240': ['n59150336'], 'n15052836': ['n59150337'], 'n15093570': ['n59150338'], 'n15050400': ['n59150339'], 'n15095510': ['n59150340'], 'n15070233': ['n59150341'], 'n15070475': ['n59150342'], 'n15070476': ['n59150343'], 'n15073500': ['n59150344'], 'n15063716': ['n59150345'], 'n15063953': ['n59150346'], 'n15082450': ['n59150347'], 'n15079051': ['n59150348'], 'n15087900': ['n59150349'], 'n15084633': ['n59150350'], 'n15097935': ['n59150351'], 'n15071770': ['n59150352'], 'n15078167': ['n59150353'], 'n15090055': ['n59150354'], 'n15092234': ['n59150355'], 'n15079015': ['n59150356'], 'n15077075': ['n59150357'], 'n15090054': ['n59150358'], 'n15091141': ['n59150359'], 'n15081570': ['n59150360'], 'n15060875': ['n59150361'], 'n15081573': ['n59150362'], 'n15084843': ['n59150363'], 'n15085932': ['n59150364'], 'n15061960': ['n59150365'], 'n15083754': ['n59150366'], 'n15073957': ['n59150367'], 'n15050837': ['n59150368'], 'n15089298': ['n59150369'], 'n15085706': ['n59150370'], 'n15061955': ['n59150371'], 'n15057453': ['n59150372'], 'n15083762': ['n59150373'], 'n15068349': ['n59150374'], 'n15077091': ['n59150375'], 'n15082677': ['n59150376'], 'n15082434': ['n59150377'], 'n15057217': ['n59150378'], 'n15055039': ['n59150379'], 'n15058789': ['n59150380'], 'n15071307': ['n59150381'], 'n15073729': ['n59150382'], 'n15067293': ['n59150383'], 'n15095723': ['n59150384'], 'n15081314': ['n59150385'], 'n15085912': ['n59150386'], 'n15061503': ['n59150387'], 'n15061983': ['n59150388'], 'n15081791': ['n59150389'], 'n15077062': ['n59150390'], 'n15078392': ['n59150391'], 'n15059667': ['n59150392'], 'n15082884': ['n59150393'], 'n15099094': ['n59150394'], 'n15059668': ['n59150395'], 'n15082883': ['n59150396'], 'n15091126': ['n59150397'], 'n15080466': ['n59150398'], 'n15059669': ['n59150399'], 'n15094637': ['n59150400'], 'n15087095': ['n59150401'], 'n15066196': ['n59150402'], 'n15089291': ['n59150403'], 'n15089292': ['n59150404'], 'n15072612': ['n59150405'], 'n15069688': ['n59150406'], 'n15099076': ['n59150407'], 'n15082898': ['n59150408'], 'n15093532': ['n59150409'], 'n15069209': ['n59150410'], 'n15078158': ['n59150411'], 'n15092200': ['n59150412'], 'n15092442': ['n59150413'], 'n15078153': ['n59150414'], 'n15084838': ['n59150415'], 'n15059651': ['n59150416'], 'n15081323': ['n59150417'], 'n15073705': ['n59150418'], 'n15089046': ['n59150419'], 'n15073704': ['n59150420'], 'n15065099': ['n59150421'], 'n15068123': ['n59150422'], 'n15065098': ['n59150423'], 'n15068122': ['n59150424'], 'n15069692': ['n59150425'], 'n15073476': ['n59150426'], 'n15096596': ['n59150427'], 'n15076747': ['n59150428'], 'n15078926': ['n59150429'], 'n15051204': ['n59150430'], 'n15053865': ['n59150431'], 'n15073474': ['n59150432'], 'n15078925': ['n59150433'], 'n15054953': ['n59150434'], 'n15075894': ['n59150435'], 'n15050594': ['n59150436'], 'n15052530': ['n59150437'], 'n15071290': ['n59150438'], 'n15065608': ['n59150439'], 'n15096111': ['n59150440'], 'n15061248': ['n59150441'], 'n15061485': ['n59150442'], 'n15064750': ['n59150443'], 'n15081094': ['n59150444'], 'n15078929': ['n59150445'], 'n15076756': ['n59150446'], 'n15097433': ['n59150447'], 'n15077603': ['n59150448'], 'n15093074': ['n59150449'], 'n15053855': ['n59150450'], 'n15096104': ['n59150451'], 'n15096345': ['n59150452'], 'n15053611': ['n59150453'], 'n15076752': ['n59150454'], 'n15096105': ['n59150455'], 'n15052761': ['n59150456'], 'n15076996': ['n59150457'], 'n15099614': ['n59150458'], 'n15073489': ['n59150459'], 'n15076992': ['n59150460'], 'n15058071': ['n59150461'], 'n15058074': ['n59150462'], 'n15094164': ['n59150463'], 'n15087887': ['n59150464'], 'n15089824': ['n59150465'], 'n15085225': ['n59150466'], 'n15089825': ['n59150467'], 'n15059157': ['n59150468'], 'n15088973': ['n59150469'], 'n15086559': ['n59150470'], 'n15085462': ['n59150471'], 'n15086551': ['n59150472'], 'n15086550': ['n59150473'], 'n15075634': ['n59150474'], 'n15095243': ['n59150475'], 'n15093062': ['n59150476'], 'n15071275': ['n59150477'], 'n15094158': ['n59150478'], 'n15098515': ['n59150479'], 'n15084108': ['n59150480'], 'n15075870': ['n59150481'], 'n15071030': ['n59150482'], 'n15071271': ['n59150483'], 'n15059188': ['n59150484'], 'n15082164': ['n59150485'], 'n15060176': ['n59150486'], 'n15065625': ['n59150487'], 'n15066956': ['n59150488'], 'n15083016': ['n59150489'], 'n15085436': ['n59150490'], 'n15088702': ['n59150491'], 'n15055826': ['n59150492'], 'n15077818': ['n59150493'], 'n15053879': ['n59150494'], 'n15051217': ['n59150495'], 'n15055817': ['n59150496'], 'n15098741': ['n59150497'], 'n15098502': ['n59150498'], 'n15076979': ['n59150499'], 'n15075883': ['n59150500'], 'n15094149': ['n59150501'], 'n15075881': ['n59150502'], 'n15051691': ['n59150503'], 'n15089807': ['n59150504'], 'n15098740': ['n59150505'], 'n15058089': ['n59150506'], 'n15085446': ['n59150507'], 'n15082170': ['n59150508'], 'n15055812': ['n59150509'], 'n15085682': ['n59150510'], 'n15086530': ['n59150511'], 'n15087860': ['n59150512'], 'n15085680': ['n59150513'], 'n15054969': ['n59150514'], 'n15051887': ['n59150515'], 'n15052976': ['n59150516'], 'n15097400': ['n59150517'], 'n15068085': ['n59150518'], 'n15075856': ['n59150519'], 'n15097885': ['n59150520'], 'n15094137': ['n59150521'], 'n15050311': ['n59150522'], 'n15093049': ['n59150523'], 'n15094371': ['n59150524'], 'n15086743': ['n59150525'], 'n15064714': ['n59150526'], 'n15082386': ['n59150527'], 'n15072109': ['n59150528'], 'n15072112': ['n59150529'], 'n15091095': ['n59150530'], 'n15095453': ['n59150531'], 'n15051632': ['n59150532'], 'n15098961': ['n59150533'], 'n15072594': ['n59150534'], 'n15099813': ['n59150535'], 'n15083009': ['n59150536'], 'n15060108': ['n59150537'], 'n15093278': ['n59150538'], 'n15061679': ['n59150539'], 'n15072593': ['n59150540'], 'n15061438': ['n59150541'], 'n15056095': ['n59150542'], 'n15056088': ['n59150543'], 'n15056089': ['n59150544'], 'n15064944': ['n59150545'], 'n15058268': ['n59150546'], 'n15082399': ['n59150547'], 'n15087604': ['n59150548'], 'n15059599': ['n59150549'], 'n15060102': ['n59150550'], 'n15094129': ['n59150551'], 'n15050307': ['n59150552'], 'n15093020': ['n59150553'], 'n15052999': ['n59150554'], 'n15073415': ['n59150555'], 'n15094114': ['n59150556'], 'n15097866': ['n59150557'], 'n15074503': ['n59150558'], 'n15097865': ['n59150559'], 'n15098712': ['n59150560'], 'n15052992': ['n59150561'], 'n15096779': ['n59150562'], 'n15050570': ['n59150563'], 'n15063888': ['n59150564'], 'n15062551': ['n59150565'], 'n15063641': ['n59150566'], 'n15064730': ['n59150567'], 'n15062554': ['n59150568'], 'n15074747': ['n59150569'], 'n15069162': ['n59150570'], 'n15053848': ['n59150571'], 'n15075843': ['n59150572'], 'n15071241': ['n59150573'], 'n15073420': ['n59150574'], 'n15094343': ['n59150575'], 'n15051655': ['n59150576'], 'n15050322': ['n59150577'], 'n15050321': ['n59150578'], 'n15098703': ['n59150579'], 'n15082139': ['n59150580'], 'n15088917': ['n59150581'], 'n15084559': ['n59150582'], 'n15064964': ['n59150583'], 'n15058288': ['n59150584'], 'n15086730': ['n59150585'], 'n15087825': ['n59150586'], 'n15086976': ['n59150587'], 'n15088912': ['n59150588'], 'n15063873': ['n59150589'], 'n15062781': ['n59150590'], 'n15075606': ['n59150591'], 'n15081040': ['n59150592'], 'n15078769': ['n59150593'], 'n15074162': ['n59150594'], 'n15076342': ['n59150595'], 'n15075254': ['n59150596'], 'n15076343': ['n59150597'], 'n15080844': ['n59150598'], 'n15054791': ['n59150599'], 'n15099465': ['n59150600'], 'n15099221': ['n59150601'], 'n15064118': ['n59150602'], 'n15064119': ['n59150603'], 'n15062177': ['n59150604'], 'n15067866': ['n59150605'], 'n15089898': ['n59150606'], 'n15068710': ['n59150607'], 'n15064353': ['n59150608'], 'n15086381': ['n59150609'], 'n15086386': ['n59150610'], 'n15088565': ['n59150611'], 'n15091508': ['n59150612'], 'n15087473': ['n59150613'], 'n15098128': ['n59150614'], 'n15076114': ['n59150615'], 'n15052126': ['n59150616'], 'n15079866': ['n59150617'], 'n15099693': ['n59150618'], 'n15095095': ['n59150619'], 'n15050182': ['n59150620'], 'n15075261': ['n59150621'], 'n15080616': ['n59150622'], 'n15097035': ['n59150623'], 'n15099452': ['n59150624'], 'n15066769': ['n59150625'], 'n15068946': ['n59150626'], 'n15055870': ['n59150627'], 'n15064102': ['n59150628'], 'n15091973': ['n59150629'], 'n15057811': ['n59150630'], 'n15064101': ['n59150631'], 'n15089665': ['n59150632'], 'n15056967': ['n59150633'], 'n15087001': ['n59150634'], 'n15089660': ['n59150635'], 'n15063491': ['n59150636'], 'n15087242': ['n59150637'], 'n15054337': ['n59150638'], 'n15099208': ['n59150639'], 'n15074140': ['n59150640'], 'n15099681': ['n59150641'], 'n15080821': ['n59150642'], 'n15073055': ['n59150643'], 'n15074143': ['n59150644'], 'n15075471': ['n59150645'], 'n15076560': ['n59150646'], 'n15077890': ['n59150647'], 'n15098112': ['n59150648'], 'n15056991': ['n59150649'], 'n15092812': ['n59150650'], 'n15056751': ['n59150651'], 'n15086128': ['n59150652'], 'n15068975': ['n59150653'], 'n15063285': ['n59150654'], 'n15088300': ['n59150655'], 'n15088542': ['n59150656'], 'n15099677': ['n59150657'], 'n15053237': ['n59150658'], 'n15051059': ['n59150659'], 'n15054324': ['n59150660'], 'n15051055': ['n59150661'], 'n15051297': ['n59150662'], 'n15051052': ['n59150663'], 'n15077421': ['n59150664'], 'n15098581': ['n59150665'], 'n15090869': ['n59150666'], 'n15092804': ['n59150667'], 'n15087467': ['n59150668'], 'n15055650': ['n59150669'], 'n15066789': ['n59150670'], 'n15068965': ['n59150671'], 'n15066787': ['n59150672'], 'n15058921': ['n59150673'], 'n15089642': ['n59150674'], 'n15055657': ['n59150675'], 'n15065210': ['n59150676'], 'n15063270': ['n59150677'], 'n15092809': ['n59150678'], 'n15055417': ['n59150679'], 'n15056506': ['n59150680'], 'n15061092': ['n59150681'], 'n15088795': ['n59150682'], 'n15053669': ['n59150683'], 'n15076306': ['n59150684'], 'n15076307': ['n59150685'], 'n15099665': ['n59150686'], 'n15051485': ['n59150687'], 'n15051242': ['n59150688'], 'n15053661': ['n59150689'], 'n15054750': ['n59150690'], 'n15075211': ['n59150691'], 'n15087678': ['n59150692'], 'n15065649': ['n59150693'], 'n15064312': ['n59150694'], 'n15066975': ['n59150695'], 'n15063466': ['n59150696'], 'n15091941': ['n59150697'], 'n15061040': ['n59150698'], 'n15063460': ['n59150699'], 'n15086101': ['n59150700'], 'n15084165': ['n59150701'], 'n15086582': ['n59150702'], 'n15090618': ['n59150703'], 'n15054505': ['n59150704'], 'n15096387': ['n59150705'], 'n15076796': ['n59150706'], 'n15051477': ['n59150707'], 'n15095291': ['n59150708'], 'n15099652': ['n59150709'], 'n15080815': ['n59150710'], 'n15088778': ['n59150711'], 'n15064788': ['n59150712'], 'n15088776': ['n59150713'], 'n15063459': ['n59150714'], 'n15068902': ['n59150715'], 'n15086598': ['n59150716'], 'n15089864': ['n59150717'], 'n15066721': ['n59150718'], 'n15065630': ['n59150719'], 'n15089620': ['n59150720'], 'n15060183': ['n59150721'], 'n15074103': ['n59150722'], 'n15096379': ['n59150723'], 'n15075439': ['n59150724'], 'n15098556': ['n59150725'], 'n15054777': ['n59150726'], 'n15075434': ['n59150727'], 'n15099405': ['n59150728'], 'n15073258': ['n59150729'], 'n15052594': ['n59150730'], 'n15054531': ['n59150731'], 'n15078940': ['n59150732'], 'n15095281': ['n59150733'], 'n15050174': ['n59150734'], 'n15096375': ['n59150735'], 'n15087656': ['n59150736'], 'n15084389': ['n59150737'], 'n15067604': ['n59150738'], 'n15066513': ['n59150739'], 'n15055864': ['n59150740'], 'n15078707': ['n59150741'], 'n15064570': ['n59150742'], 'n15083295': ['n59150743'], 'n15097213': ['n59150744'], 'n15074356': ['n59150745'], 'n15054768': ['n59150746'], 'n15093091': ['n59150747'], 'n15099637': ['n59150748'], 'n15075683': ['n59150749'], 'n15096362': ['n59150750'], 'n15067837': ['n59150751'], 'n15068925': ['n59150752'], 'n15088511': ['n59150753'], 'n15066986': ['n59150754'], 'n15061056': ['n59150755'], 'n15065898': ['n59150756'], 'n15068922': ['n59150757'], 'n15088759': ['n59150758'], 'n15062389': ['n59150759'], 'n15055852': ['n59150760'], 'n15077629': ['n59150761'], 'n15084154': ['n59150762'], 'n15088991': ['n59150763'], 'n15071851': ['n59150764'], 'n15078246': ['n59150765'], 'n15054273': ['n59150766'], 'n15052093': ['n59150767'], 'n15099189': ['n59150768'], 'n15078492': ['n59150769'], 'n15067349': ['n59150770'], 'n15056698': ['n59150771'], 'n15057302': ['n59150772'], 'n15067587': ['n59150773'], 'n15094736': ['n59150774'], 'n15094978': ['n59150775'], 'n15067585': ['n59150776'], 'n15096916': ['n59150777'], 'n15089374': ['n59150778'], 'n15066251': ['n59150779'], 'n15070532': ['n59150780'], 'n15070775': ['n59150781'], 'n15069545': ['n59150782'], 'n15069304': ['n59150783'], 'n15094964': ['n59150784'], 'n15079588': ['n59150785'], 'n15053177': ['n59150786'], 'n15060947': ['n59150787'], 'n15060707': ['n59150788'], 'n15098086': ['n59150789'], 'n15066249': ['n59150790'], 'n15055597': ['n59150791'], 'n15083600': ['n59150792'], 'n15099181': ['n59150793'], 'n15055116': ['n59150794'], 'n15066241': ['n59150795'], 'n15072717': ['n59150796'], 'n15050900': ['n59150797'], 'n15065191': ['n59150798'], 'n15069512': ['n59150799'], 'n15094711': ['n59150800'], 'n15079313': ['n59150801'], 'n15079556': ['n59150802'], 'n15061829': ['n59150803'], 'n15082726': ['n59150804'], 'n15059982': ['n59150805'], 'n15076292': ['n59150806'], 'n15059744': ['n59150807'], 'n15089116': ['n59150808'], 'n15080783': ['n59150809'], 'n15081872': ['n59150810'], 'n15080784': ['n59150811'], 'n15059501': ['n59150812'], 'n15091204': ['n59150813'], 'n15093623': ['n59150814'], 'n15071835': ['n59150815'], 'n15088263': ['n59150816'], 'n15067124': ['n59150817'], 'n15067123': ['n59150818'], 'n15058419': ['n59150819'], 'n15066031': ['n59150820'], 'n15088023': ['n59150821'], 'n15089595': ['n59150822'], 'n15069763': ['n59150823'], 'n15082976': ['n59150824'], 'n15099399': ['n59150825'], 'n15083829': ['n59150826'], 'n15057794': ['n59150827'], 'n15093854': ['n59150828'], 'n15058404': ['n59150829'], 'n15066262': ['n59150830'], 'n15069978': ['n59150831'], 'n15075171': ['n59150832'], 'n15090338': ['n59150833'], 'n15067307': ['n59150834'], 'n15056652': ['n59150835'], 'n15065128': ['n59150836'], 'n15055564': ['n59150837'], 'n15068877': ['n59150838'], 'n15090576': ['n59150839'], 'n15065368': ['n59150840'], 'n15087398': ['n59150841'], 'n15059926': ['n59150842'], 'n15089330': ['n59150843'], 'n15052044': ['n59150844'], 'n15091894': ['n59150845'], 'n15052287': ['n59150846'], 'n15081623': ['n59150847'], 'n15079300': ['n59150848'], 'n15092981': ['n59150849'], 'n15098284': ['n59150850'], 'n15082716': ['n59150851'], 'n15077360': ['n59150852'], 'n15098044': ['n59150853'], 'n15078693': ['n59150854'], 'n15067539': ['n59150855'], 'n15092746': ['n59150856'], 'n15080772': ['n59150857'], 'n15066203': ['n59150858'], 'n15056401': ['n59150859'], 'n15068864': ['n59150860'], 'n15070976': ['n59150861'], 'n15056403': ['n59150862'], 'n15064023': ['n59150863'], 'n15072910': ['n59150864'], 'n15065350': ['n59150865'], 'n15088015': ['n59150866'], 'n15077337': ['n59150867'], 'n15053166': ['n59150868'], 'n15079996': ['n59150869'], 'n15098031': ['n59150870'], 'n15080988': ['n59150871'], 'n15054251': ['n59150872'], 'n15054493': ['n59150873'], 'n15082924': ['n59150874'], 'n15053160': ['n59150875'], 'n15056670': ['n59150876'], 'n15057761': ['n59150877'], 'n15068417': ['n59150878'], 'n15066239': ['n59150879'], 'n15065389': ['n59150880'], 'n15056434': ['n59150881'], 'n15055348': ['n59150882'], 'n15066474': ['n59150883'], 'n15089555': ['n59150884'], 'n15065141': ['n59150885'], 'n15064053': ['n59150886'], 'n15086044': ['n59150887'], 'n15097179': ['n59150888'], 'n15060924': ['n59150889'], 'n15096082': ['n59150890'], 'n15077100': ['n59150891'], 'n15077584': ['n59150892'], 'n15090542': ['n59150893'], 'n15060926': ['n59150894'], 'n15069725': ['n59150895'], 'n15060920': ['n59150896'], 'n15080999': ['n59150897'], 'n15075161': ['n59150898'], 'n15052060': ['n59150899'], 'n15054480': ['n59150900'], 'n15077340': ['n59150901'], 'n15094909': ['n59150902'], 'n15057752': ['n59150903'], 'n15056663': ['n59150904'], 'n15055334': ['n59150905'], 'n15059935': ['n59150906'], 'n15065134': ['n59150907'], 'n15087140': ['n59150908'], 'n15064281': ['n59150909'], 'n15093819': ['n59150910'], 'n15086055': ['n59150911'], 'n15072541': ['n59150912'], 'n15075811': ['n59150913'], 'n15094334': ['n59150914'], 'n15095665': ['n59150915'], 'n15072786': ['n59150916'], 'n15097846': ['n59150917'], 'n15085859': ['n59150918'], 'n15093006': ['n59150919'], 'n15072780': ['n59150920'], 'n15078087': ['n59150921'], 'n15062979': ['n59150922'], 'n15079182': ['n59150923'], 'n15057386': ['n59150924'], 'n15059321': ['n59150925'], 'n15086945': ['n59150926'], 'n15060550': ['n59150927'], 'n15082347': ['n59150928'], 'n15086701': ['n59150929'], 'n15070127': ['n59150930'], 'n15050517': ['n59150931'], 'n15066097': ['n59150932'], 'n15072553': ['n59150933'], 'n15076911': ['n59150934'], 'n15070372': ['n59150935'], 'n15070379': ['n59150936'], 'n15071226': ['n59150937'], 'n15098924': ['n59150938'], 'n15073643': ['n59150939'], 'n15093235': ['n59150940'], 'n15062725': ['n59150941'], 'n15079189': ['n59150942'], 'n15056052': ['n59150943'], 'n15060784': ['n59150944'], 'n15057375': ['n59150945'], 'n15058226': ['n59150946'], 'n15071469': ['n59150947'], 'n15074737': ['n59150948'], 'n15050748': ['n59150949'], 'n15068264': ['n59150950'], 'n15082591': ['n59150951'], 'n15072761': ['n59150952'], 'n15095640': ['n59150953'], 'n15053800': ['n59150954'], 'n15094312': ['n59150955'], 'n15073854': ['n59150956'], 'n15097821': ['n59150957'], 'n15071433': ['n59150958'], 'n15058020': ['n59150959'], 'n15061425': ['n59150960'], 'n15083654': ['n59150961'], 'n15084984': ['n59150962'], 'n15092139': ['n59150963'], 'n15071438': ['n59150964'], 'n15071439': ['n59150965'], 'n15098919': ['n59150966'], 'n15075801': ['n59150967'], 'n15094541': ['n59150968'], 'n15073623': ['n59150969'], 'n15086936': ['n59150970'], 'n15070590': ['n59150971'], 'n15062505': ['n59150972'], 'n15079167': ['n59150973'], 'n15063838': ['n59150974'], 'n15094780': ['n59150975'], 'n15057154': ['n59150976'], 'n15080394': ['n59150977'], 'n15060566': ['n59150978'], 'n15080398': ['n59150979'], 'n15081004': ['n59150980'], 'n15093216': ['n59150981'], 'n15058009': ['n59150982'], 'n15096726': ['n59150983'], 'n15050529': ['n59150984'], 'n15096967': ['n59150985'], 'n15098906': ['n59150986'], 'n15073625': ['n59150987'], 'n15051858': ['n59150988'], 'n15094309': ['n59150989'], 'n15068044': ['n59150990'], 'n15051615': ['n59150991'], 'n15074718': ['n59150992'], 'n15093680': ['n59150993'], 'n15071651': ['n59150994'], 'n15079139': ['n59150995'], 'n15085815': ['n59150996'], 'n15060518': ['n59150997'], 'n15093203': ['n59150998'], 'n15080361': ['n59150999'], 'n15057343': ['n59151000'], 'n15055165': ['n59151001'], 'n15082785': ['n59151002'], 'n15080365': ['n59151003'], 'n15081454': ['n59151004'], 'n15057589': ['n59151005'], 'n15068239': ['n59151006'], 'n15094779': ['n59151007'], 'n15066299': ['n59151008'], 'n15067383': ['n59151009'], 'n15069340': ['n59151010'], 'n15092580': ['n59151011'], 'n15090160': ['n59151012'], 'n15094765': ['n59151013'], 'n15060502': ['n59151014'], 'n15060742': ['n59151015'], 'n15055156': ['n59151016'], 'n15057333': ['n59151017'], 'n15081223': ['n59151018'], 'n15085823': ['n59151019'], 'n15083887': ['n59151020'], 'n15086910': ['n59151021'], 'n15055159': ['n59151022'], 'n15059757': ['n59151023'], 'n15068020': ['n59151024'], 'n15072721': ['n59151025'], 'n15084946': ['n59151026'], 'n15062719': ['n59151027'], 'n15080106': ['n59151028'], 'n15093663': ['n59151029'], 'n15062714': ['n59151030'], 'n15057363': ['n59151031'], 'n15058211': ['n59151032'], 'n15057366': ['n59151033'], 'n15058213': ['n59151034'], 'n15057127': ['n59151035'], 'n15059786': ['n59151036'], 'n15082765': ['n59151037'], 'n15059787': ['n59151038'], 'n15092336': ['n59151039'], 'n15059548': ['n59151040'], 'n15088063': ['n59151041'], 'n15050979': ['n59151042'], 'n15070792': ['n59151043'], 'n15072971': ['n59151044'], 'n15061851': ['n59151045'], 'n15056264': ['n59151046'], 'n15081683': ['n59151047'], 'n15081444': ['n59151048'], 'n15084951': ['n59151049'], 'n15093657': ['n59151050'], 'n15066069': ['n59151051'], 'n15067158': ['n59151052'], 'n15096926': ['n59151053'], 'n15089164': ['n59151054'], 'n15117019': ['n59151055'], 'n15101906': ['n59151056'], 'n15117016': ['n59151057'], 'n15117015': ['n59151058'], 'n15117499': ['n59151059'], 'n15117018': ['n59151060'], 'n15117017': ['n59151061'], 'n15117023': ['n59151062'], 'n15117022': ['n59151063'], 'n15117025': ['n59151064'], 'n15117024': ['n59151065'], 'n15116172': ['n59151066'], 'n15117021': ['n59151067'], 'n15117020': ['n59151068'], 'n15106358': ['n59151069'], 'n15109885': ['n59151070'], 'n15105026': ['n59151071'], 'n15110858': ['n59151072'], 'n15110859': ['n59151073'], 'n15117009': ['n59151074'], 'n15117008': ['n59151075'], 'n15117005': ['n59151076'], 'n15117489': ['n59151077'], 'n15117004': ['n59151078'], 'n15117488': ['n59151079'], 'n15117007': ['n59151080'], 'n15117006': ['n59151081'], 'n15117012': ['n59151082'], 'n15117496': ['n59151083'], 'n15117011': ['n59151084'], 'n15117495': ['n59151085'], 'n15117014': ['n59151086'], 'n15117498': ['n59151087'], 'n15117013': ['n59151088'], 'n15117497': ['n59151089'], 'n15117492': ['n59151090'], 'n15117491': ['n59151091'], 'n15117010': ['n59151092'], 'n15117494': ['n59151093'], 'n15117493': ['n59151094'], 'n15117490': ['n59151095'], 'n15108566': ['n59151096'], 'n15108564': ['n59151097'], 'n15110405': ['n59151098'], 'n15117038': ['n59151099'], 'n15117037': ['n59151100'], 'n15117039': ['n59151101'], 'n15117045': ['n59151102'], 'n15117044': ['n59151103'], 'n15117047': ['n59151104'], 'n15117046': ['n59151105'], 'n15117041': ['n59151106'], 'n15117040': ['n59151107'], 'n15117043': ['n59151108'], 'n15117042': ['n59151109'], 'n15108556': ['n59151110'], 'n15107242': ['n59151111'], 'n15117027': ['n59151112'], 'n15117026': ['n59151113'], 'n15117029': ['n59151114'], 'n15117028': ['n59151115'], 'n15117034': ['n59151116'], 'n15117033': ['n59151117'], 'n15117036': ['n59151118'], 'n15117035': ['n59151119'], 'n15117030': ['n59151120'], 'n15117032': ['n59151121'], 'n15117031': ['n59151122'], 'n15108568': ['n59151123'], 'n15112602': ['n59151124'], 'n15109439': ['n59151125'], 'n15108581': ['n59151126'], 'n15117059': ['n59151127'], 'n15117067': ['n59151128'], 'n15101947': ['n59151129'], 'n15117066': ['n59151130'], 'n15117069': ['n59151131'], 'n15117068': ['n59151132'], 'n15117063': ['n59151133'], 'n15117062': ['n59151134'], 'n15117065': ['n59151135'], 'n15117064': ['n59151136'], 'n15117061': ['n59151137'], 'n15117060': ['n59151138'], 'n15110410': ['n59151139'], 'n15117049': ['n59151140'], 'n15117048': ['n59151141'], 'n15110893': ['n59151142'], 'n15117056': ['n59151143'], 'n15117055': ['n59151144'], 'n15117058': ['n59151145'], 'n15117057': ['n59151146'], 'n15117052': ['n59151147'], 'n15117051': ['n59151148'], 'n15117054': ['n59151149'], 'n15117053': ['n59151150'], 'n15117050': ['n59151151'], 'n15117089': ['n59151152'], 'n15117088': ['n59151153'], 'n15102818': ['n59151154'], 'n15117085': ['n59151155'], 'n15117084': ['n59151156'], 'n15117087': ['n59151157'], 'n15117086': ['n59151158'], 'n15117081': ['n59151159'], 'n15117080': ['n59151160'], 'n15117083': ['n59151161'], 'n15117082': ['n59151162'], 'n15117078': ['n59151163'], 'n15117077': ['n59151164'], 'n15117079': ['n59151165'], 'n15102823': ['n59151166'], 'n15117074': ['n59151167'], 'n15117073': ['n59151168'], 'n15117076': ['n59151169'], 'n15117075': ['n59151170'], 'n15117070': ['n59151171'], 'n15117072': ['n59151172'], 'n15117071': ['n59151173'], 'n15108127': ['n59151174'], 'n15113964': ['n59151175'], 'n15117099': ['n59151176'], 'n15117096': ['n59151177'], 'n15117095': ['n59151178'], 'n15117098': ['n59151179'], 'n15117097': ['n59151180'], 'n15117092': ['n59151181'], 'n15109473': ['n59151182'], 'n15117091': ['n59151183'], 'n15117094': ['n59151184'], 'n15117093': ['n59151185'], 'n15117090': ['n59151186'], 'n15110007': ['n59151187'], 'n15111330': ['n59151188'], 'n15113511': ['n59151189'], 'n15114836': ['n59151190'], 'n15103719': ['n59151191'], 'n15101536': ['n59151192'], 'n15108181': ['n59151193'], 'n15111360': ['n59151194'], 'n15110492': ['n59151195'], 'n15102400': ['n59151196'], 'n15117919': ['n59151197'], 'n15117918': ['n59151198'], 'n15112227': ['n59151199'], 'n15117915': ['n59151200'], 'n15117914': ['n59151201'], 'n15117917': ['n59151202'], 'n15117916': ['n59151203'], 'n15117911': ['n59151204'], 'n15115733': ['n59151205'], 'n15117910': ['n59151206'], 'n15117913': ['n59151207'], 'n15117912': ['n59151208'], 'n15100238': ['n59151209'], 'n15117908': ['n59151210'], 'n15117907': ['n59151211'], 'n15117909': ['n59151212'], 'n15117904': ['n59151213'], 'n15117903': ['n59151214'], 'n15117906': ['n59151215'], 'n15117905': ['n59151216'], 'n15117900': ['n59151217'], 'n15117902': ['n59151218'], 'n15117901': ['n59151219'], 'n15115720': ['n59151220'], 'n15105937': ['n59151221'], 'n15117937': ['n59151222'], 'n15117936': ['n59151223'], 'n15117939': ['n59151224'], 'n15117938': ['n59151225'], 'n15117933': ['n59151226'], 'n15117932': ['n59151227'], 'n15114425': ['n59151228'], 'n15117935': ['n59151229'], 'n15117934': ['n59151230'], 'n15115751': ['n59151231'], 'n15116600': ['n59151232'], 'n15117931': ['n59151233'], 'n15117930': ['n59151234'], 'n15102438': ['n59151235'], 'n15110072': ['n59151236'], 'n15104613': ['n59151237'], 'n15100252': ['n59151238'], 'n15117929': ['n59151239'], 'n15117926': ['n59151240'], 'n15117925': ['n59151241'], 'n15117928': ['n59151242'], 'n15117927': ['n59151243'], 'n15117922': ['n59151244'], 'n15117921': ['n59151245'], 'n15117924': ['n59151246'], 'n15117923': ['n59151247'], 'n15117920': ['n59151248'], 'n15106808': ['n59151249'], 'n15103779': ['n59151250'], 'n15117959': ['n59151251'], 'n15117958': ['n59151252'], 'n15117955': ['n59151253'], 'n15117954': ['n59151254'], 'n15117957': ['n59151255'], 'n15117956': ['n59151256'], 'n15117951': ['n59151257'], 'n15117950': ['n59151258'], 'n15117953': ['n59151259'], 'n15117952': ['n59151260'], 'n15117960': ['n59151261'], 'n15106811': ['n59151262'], 'n15100293': ['n59151263'], 'n15117948': ['n59151264'], 'n15117947': ['n59151265'], 'n15117949': ['n59151266'], 'n15117944': ['n59151267'], 'n15117943': ['n59151268'], 'n15117946': ['n59151269'], 'n15117945': ['n59151270'], 'n15117940': ['n59151271'], 'n15117942': ['n59151272'], 'n15117941': ['n59151273'], 'n15117977': ['n59151274'], 'n15117976': ['n59151275'], 'n15117979': ['n59151276'], 'n15117978': ['n59151277'], 'n15117973': ['n59151278'], 'n15113138': ['n59151279'], 'n15117972': ['n59151280'], 'n15117975': ['n59151281'], 'n15117974': ['n59151282'], 'n15117980': ['n59151283'], 'n15117982': ['n59151284'], 'n15117981': ['n59151285'], 'n15112295': ['n59151286'], 'n15105509': ['n59151287'], 'n15112293': ['n59151288'], 'n15104670': ['n59151289'], 'n15115307': ['n59151290'], 'n15117969': ['n59151291'], 'n15117966': ['n59151292'], 'n15117965': ['n59151293'], 'n15116637': ['n59151294'], 'n15117968': ['n59151295'], 'n15117967': ['n59151296'], 'n15117962': ['n59151297'], 'n15117961': ['n59151298'], 'n15117964': ['n59151299'], 'n15117963': ['n59151300'], 'n15117971': ['n59151301'], 'n15117970': ['n59151302'], 'n15104669': ['n59151303'], 'n15104663': ['n59151304'], 'n15105510': ['n59151305'], 'n15106861': ['n59151306'], 'n15117519': ['n59151307'], 'n15117518': ['n59151308'], 'n15117515': ['n59151309'], 'n15117999': ['n59151310'], 'n15117514': ['n59151311'], 'n15117998': ['n59151312'], 'n15117517': ['n59151313'], 'n15117516': ['n59151314'], 'n15117511': ['n59151315'], 'n15117995': ['n59151316'], 'n15117510': ['n59151317'], 'n15117994': ['n59151318'], 'n15117513': ['n59151319'], 'n15117997': ['n59151320'], 'n15117512': ['n59151321'], 'n15117996': ['n59151322'], 'n15117520': ['n59151323'], 'n15106855': ['n59151324'], 'n15117508': ['n59151325'], 'n15117507': ['n59151326'], 'n15117509': ['n59151327'], 'n15117504': ['n59151328'], 'n15117988': ['n59151329'], 'n15117503': ['n59151330'], 'n15117987': ['n59151331'], 'n15117506': ['n59151332'], 'n15117505': ['n59151333'], 'n15117989': ['n59151334'], 'n15117500': ['n59151335'], 'n15117984': ['n59151336'], 'n15117983': ['n59151337'], 'n15117502': ['n59151338'], 'n15117986': ['n59151339'], 'n15117501': ['n59151340'], 'n15117985': ['n59151341'], 'n15117991': ['n59151342'], 'n15117990': ['n59151343'], 'n15117993': ['n59151344'], 'n15117992': ['n59151345'], 'n15113153': ['n59151346'], 'n15117537': ['n59151347'], 'n15117536': ['n59151348'], 'n15117539': ['n59151349'], 'n15117538': ['n59151350'], 'n15117533': ['n59151351'], 'n15117532': ['n59151352'], 'n15117535': ['n59151353'], 'n15117534': ['n59151354'], 'n15117540': ['n59151355'], 'n15117542': ['n59151356'], 'n15117541': ['n59151357'], 'n15113185': ['n59151358'], 'n15102037': ['n59151359'], 'n15107740': ['n59151360'], 'n15117529': ['n59151361'], 'n15117526': ['n59151362'], 'n15117525': ['n59151363'], 'n15117528': ['n59151364'], 'n15117527': ['n59151365'], 'n15117522': ['n59151366'], 'n15117521': ['n59151367'], 'n15117524': ['n59151368'], 'n15113169': ['n59151369'], 'n15117523': ['n59151370'], 'n15117531': ['n59151371'], 'n15117530': ['n59151372'], 'n15106405': ['n59151373'], 'n15106404': ['n59151374'], 'n15105573': ['n59151375'], 'n15117559': ['n59151376'], 'n15117558': ['n59151377'], 'n15117555': ['n59151378'], 'n15117554': ['n59151379'], 'n15117557': ['n59151380'], 'n15117556': ['n59151381'], 'n15117562': ['n59151382'], 'n15117561': ['n59151383'], 'n15117564': ['n59151384'], 'n15117563': ['n59151385'], 'n15117560': ['n59151386'], 'n15104254': ['n59151387'], 'n15117548': ['n59151388'], 'n15117547': ['n59151389'], 'n15117549': ['n59151390'], 'n15117544': ['n59151391'], 'n15117543': ['n59151392'], 'n15117546': ['n59151393'], 'n15117545': ['n59151394'], 'n15117551': ['n59151395'], 'n15117550': ['n59151396'], 'n15117553': ['n59151397'], 'n15117552': ['n59151398'], 'n15110918': ['n59151399'], 'n15109958': ['n59151400'], 'n15117577': ['n59151401'], 'n15115399': ['n59151402'], 'n15117576': ['n59151403'], 'n15117579': ['n59151404'], 'n15117578': ['n59151405'], 'n15117100': ['n59151406'], 'n15117584': ['n59151407'], 'n15116252': ['n59151408'], 'n15117583': ['n59151409'], 'n15116255': ['n59151410'], 'n15117102': ['n59151411'], 'n15117586': ['n59151412'], 'n15117101': ['n59151413'], 'n15117585': ['n59151414'], 'n15117580': ['n59151415'], 'n15117582': ['n59151416'], 'n15117581': ['n59151417'], 'n15107767': ['n59151418'], 'n15104275': ['n59151419'], 'n15105123': ['n59151420'], 'n15107300': ['n59151421'], 'n15117569': ['n59151422'], 'n15117566': ['n59151423'], 'n15117565': ['n59151424'], 'n15117568': ['n59151425'], 'n15117567': ['n59151426'], 'n15117573': ['n59151427'], 'n15116241': ['n59151428'], 'n15117572': ['n59151429'], 'n15117575': ['n59151430'], 'n15117574': ['n59151431'], 'n15117571': ['n59151432'], 'n15117570': ['n59151433'], 'n15111811': ['n59151434'], 'n15109978': ['n59151435'], 'n15111810': ['n59151436'], 'n15117119': ['n59151437'], 'n15117118': ['n59151438'], 'n15117115': ['n59151439'], 'n15117599': ['n59151440'], 'n15117114': ['n59151441'], 'n15117598': ['n59151442'], 'n15117117': ['n59151443'], 'n15117116': ['n59151444'], 'n15117122': ['n59151445'], 'n15117121': ['n59151446'], 'n15117124': ['n59151447'], 'n15117123': ['n59151448'], 'n15117120': ['n59151449'], 'n15102099': ['n59151450'], 'n15108635': ['n59151451'], 'n15106473': ['n59151452'], 'n15117108': ['n59151453'], 'n15117107': ['n59151454'], 'n15117109': ['n59151455'], 'n15117104': ['n59151456'], 'n15117588': ['n59151457'], 'n15117103': ['n59151458'], 'n15117587': ['n59151459'], 'n15117106': ['n59151460'], 'n15117105': ['n59151461'], 'n15117589': ['n59151462'], 'n15117111': ['n59151463'], 'n15117595': ['n59151464'], 'n15117110': ['n59151465'], 'n15117594': ['n59151466'], 'n15117113': ['n59151467'], 'n15117597': ['n59151468'], 'n15117112': ['n59151469'], 'n15117596': ['n59151470'], 'n15117591': ['n59151471'], 'n15117590': ['n59151472'], 'n15117593': ['n59151473'], 'n15117592': ['n59151474'], 'n15118229': ['n59151475'], 'n15117379': ['n59151476'], 'n15118227': ['n59151477'], 'n15117378': ['n59151478'], 'n15118228': ['n59151479'], 'n15118225': ['n59151480'], 'n15118226': ['n59151481'], 'n15117386': ['n59151482'], 'n15118234': ['n59151483'], 'n15117385': ['n59151484'], 'n15118235': ['n59151485'], 'n15117388': ['n59151486'], 'n15118232': ['n59151487'], 'n15117387': ['n59151488'], 'n15118233': ['n59151489'], 'n15117382': ['n59151490'], 'n15118230': ['n59151491'], 'n15117381': ['n59151492'], 'n15118231': ['n59151493'], 'n15117384': ['n59151494'], 'n15117383': ['n59151495'], 'n15117380': ['n59151496'], 'n15108413': ['n59151497'], 'n15109744': ['n59151498'], 'n15108898': ['n59151499'], 'n15108430': ['n59151500'], 'n15104074': ['n59151501'], 'n15118218': ['n59151502'], 'n15118219': ['n59151503'], 'n15117368': ['n59151504'], 'n15118216': ['n59151505'], 'n15117367': ['n59151506'], 'n15118217': ['n59151507'], 'n15118214': ['n59151508'], 'n15117369': ['n59151509'], 'n15118215': ['n59151510'], 'n15117375': ['n59151511'], 'n15118223': ['n59151512'], 'n15117374': ['n59151513'], 'n15118224': ['n59151514'], 'n15117377': ['n59151515'], 'n15118221': ['n59151516'], 'n15117376': ['n59151517'], 'n15118222': ['n59151518'], 'n15117371': ['n59151519'], 'n15117370': ['n59151520'], 'n15118220': ['n59151521'], 'n15117373': ['n59151522'], 'n15117372': ['n59151523'], 'n15106270': ['n59151524'], 'n15117389': ['n59151525'], 'n15117397': ['n59151526'], 'n15117396': ['n59151527'], 'n15117399': ['n59151528'], 'n15117398': ['n59151529'], 'n15117393': ['n59151530'], 'n15117392': ['n59151531'], 'n15117395': ['n59151532'], 'n15117394': ['n59151533'], 'n15117391': ['n59151534'], 'n15117390': ['n59151535'], 'n15108463': ['n59151536'], 'n15112960': ['n59151537'], 'n15107147': ['n59151538'], 'n15110777': ['n59151539'], 'n15113803': ['n59151540'], 'n15106298': ['n59151541'], 'n15100501': ['n59151542'], 'n15101830': ['n59151543'], 'n15109798': ['n59151544'], 'n15111645': ['n59151545'], 'n15110318': ['n59151546'], 'n15112527': ['n59151547'], 'n15113857': ['n59151548'], 'n15111674': ['n59151549'], 'n15102710': ['n59151550'], 'n15108037': ['n59151551'], 'n15107183': ['n59151552'], 'n15110334': ['n59151553'], 'n15101870': ['n59151554'], 'n15114728': ['n59151555'], 'n15113870': ['n59151556'], 'n15102735': ['n59151557'], 'n15104914': ['n59151558'], 'n15109389': ['n59151559'], 'n15110355': ['n59151560'], 'n15116929': ['n59151561'], 'n15116928': ['n59151562'], 'n15116925': ['n59151563'], 'n15116924': ['n59151564'], 'n15116927': ['n59151565'], 'n15116926': ['n59151566'], 'n15116921': ['n59151567'], 'n15116920': ['n59151568'], 'n15116923': ['n59151569'], 'n15116922': ['n59151570'], 'n15110377': ['n59151571'], 'n15114730': ['n59151572'], 'n15101433': ['n59151573'], 'n15102762': ['n59151574'], 'n15116947': ['n59151575'], 'n15116946': ['n59151576'], 'n15116949': ['n59151577'], 'n15116948': ['n59151578'], 'n15116943': ['n59151579'], 'n15116942': ['n59151580'], 'n15116945': ['n59151581'], 'n15116944': ['n59151582'], 'n15114762': ['n59151583'], 'n15116941': ['n59151584'], 'n15116940': ['n59151585'], 'n15102773': ['n59151586'], 'n15116939': ['n59151587'], 'n15116936': ['n59151588'], 'n15116935': ['n59151589'], 'n15116938': ['n59151590'], 'n15116937': ['n59151591'], 'n15116932': ['n59151592'], 'n15116931': ['n59151593'], 'n15116934': ['n59151594'], 'n15116933': ['n59151595'], 'n15116930': ['n59151596'], 'n15102784': ['n59151597'], 'n15117819': ['n59151598'], 'n15116969': ['n59151599'], 'n15117816': ['n59151600'], 'n15116968': ['n59151601'], 'n15117815': ['n59151602'], 'n15117818': ['n59151603'], 'n15117817': ['n59151604'], 'n15116965': ['n59151605'], 'n15117812': ['n59151606'], 'n15116964': ['n59151607'], 'n15117811': ['n59151608'], 'n15116967': ['n59151609'], 'n15117814': ['n59151610'], 'n15116966': ['n59151611'], 'n15117813': ['n59151612'], 'n15116961': ['n59151613'], 'n15116960': ['n59151614'], 'n15116963': ['n59151615'], 'n15117810': ['n59151616'], 'n15116962': ['n59151617'], 'n15116970': ['n59151618'], 'n15100139': ['n59151619'], 'n15101467': ['n59151620'], 'n15101461': ['n59151621'], 'n15117809': ['n59151622'], 'n15117808': ['n59151623'], 'n15113448': ['n59151624'], 'n15116958': ['n59151625'], 'n15117805': ['n59151626'], 'n15116957': ['n59151627'], 'n15117804': ['n59151628'], 'n15117807': ['n59151629'], 'n15116959': ['n59151630'], 'n15117806': ['n59151631'], 'n15116954': ['n59151632'], 'n15117801': ['n59151633'], 'n15116953': ['n59151634'], 'n15117800': ['n59151635'], 'n15116956': ['n59151636'], 'n15117803': ['n59151637'], 'n15116955': ['n59151638'], 'n15117802': ['n59151639'], 'n15116950': ['n59151640'], 'n15116952': ['n59151641'], 'n15116951': ['n59151642'], 'n15104504': ['n59151643'], 'n15117838': ['n59151644'], 'n15117837': ['n59151645'], 'n15117839': ['n59151646'], 'n15116987': ['n59151647'], 'n15117834': ['n59151648'], 'n15111298': ['n59151649'], 'n15116986': ['n59151650'], 'n15117833': ['n59151651'], 'n15116989': ['n59151652'], 'n15117836': ['n59151653'], 'n15116988': ['n59151654'], 'n15117835': ['n59151655'], 'n15116983': ['n59151656'], 'n15117830': ['n59151657'], 'n15116982': ['n59151658'], 'n15116985': ['n59151659'], 'n15117832': ['n59151660'], 'n15116984': ['n59151661'], 'n15117831': ['n59151662'], 'n15116990': ['n59151663'], 'n15116992': ['n59151664'], 'n15116991': ['n59151665'], 'n15101005': ['n59151666'], 'n15117827': ['n59151667'], 'n15116979': ['n59151668'], 'n15117826': ['n59151669'], 'n15117829': ['n59151670'], 'n15117828': ['n59151671'], 'n15116976': ['n59151672'], 'n15117823': ['n59151673'], 'n15116975': ['n59151674'], 'n15117822': ['n59151675'], 'n15116978': ['n59151676'], 'n15117825': ['n59151677'], 'n15116977': ['n59151678'], 'n15117824': ['n59151679'], 'n15116972': ['n59151680'], 'n15116971': ['n59151681'], 'n15116974': ['n59151682'], 'n15117821': ['n59151683'], 'n15116973': ['n59151684'], 'n15117820': ['n59151685'], 'n15116981': ['n59151686'], 'n15116980': ['n59151687'], 'n15105871': ['n59151688'], 'n15117859': ['n59151689'], 'n15117856': ['n59151690'], 'n15117855': ['n59151691'], 'n15117858': ['n59151692'], 'n15117857': ['n59151693'], 'n15117852': ['n59151694'], 'n15117851': ['n59151695'], 'n15117854': ['n59151696'], 'n15114345': ['n59151697'], 'n15117853': ['n59151698'], 'n15117861': ['n59151699'], 'n15117860': ['n59151700'], 'n15102354': ['n59151701'], 'n15117849': ['n59151702'], 'n15117848': ['n59151703'], 'n15100190': ['n59151704'], 'n15116998': ['n59151705'], 'n15117845': ['n59151706'], 'n15116997': ['n59151707'], 'n15117844': ['n59151708'], 'n15117847': ['n59151709'], 'n15116999': ['n59151710'], 'n15117846': ['n59151711'], 'n15116994': ['n59151712'], 'n15117841': ['n59151713'], 'n15116993': ['n59151714'], 'n15117840': ['n59151715'], 'n15116996': ['n59151716'], 'n15117843': ['n59151717'], 'n15116995': ['n59151718'], 'n15117842': ['n59151719'], 'n15117850': ['n59151720'], 'n15113010': ['n59151721'], 'n15105892': ['n59151722'], 'n15113036': ['n59151723'], 'n15117878': ['n59151724'], 'n15114369': ['n59151725'], 'n15117877': ['n59151726'], 'n15117879': ['n59151727'], 'n15117874': ['n59151728'], 'n15117873': ['n59151729'], 'n15117876': ['n59151730'], 'n15113037': ['n59151731'], 'n15117875': ['n59151732'], 'n15117881': ['n59151733'], 'n15117880': ['n59151734'], 'n15117883': ['n59151735'], 'n15117882': ['n59151736'], 'n15105420': ['n59151737'], 'n15108911': ['n59151738'], 'n15117867': ['n59151739'], 'n15117866': ['n59151740'], 'n15117869': ['n59151741'], 'n15112175': ['n59151742'], 'n15117868': ['n59151743'], 'n15117863': ['n59151744'], 'n15117862': ['n59151745'], 'n15117865': ['n59151746'], 'n15117864': ['n59151747'], 'n15117870': ['n59151748'], 'n15117872': ['n59151749'], 'n15117871': ['n59151750'], 'n15101054': ['n59151751'], 'n15117419': ['n59151752'], 'n15117416': ['n59151753'], 'n15117415': ['n59151754'], 'n15117899': ['n59151755'], 'n15117418': ['n59151756'], 'n15117417': ['n59151757'], 'n15114386': ['n59151758'], 'n15117412': ['n59151759'], 'n15117896': ['n59151760'], 'n15117411': ['n59151761'], 'n15117895': ['n59151762'], 'n15117414': ['n59151763'], 'n15117898': ['n59151764'], 'n15117413': ['n59151765'], 'n15117897': ['n59151766'], 'n15117421': ['n59151767'], 'n15117420': ['n59151768'], 'n15107604': ['n59151769'], 'n15105424': ['n59151770'], 'n15107601': ['n59151771'], 'n15117409': ['n59151772'], 'n15117408': ['n59151773'], 'n15117405': ['n59151774'], 'n15117889': ['n59151775'], 'n15117404': ['n59151776'], 'n15117888': ['n59151777'], 'n15117407': ['n59151778'], 'n15117406': ['n59151779'], 'n15117401': ['n59151780'], 'n15117885': ['n59151781'], 'n15117400': ['n59151782'], 'n15117884': ['n59151783'], 'n15117403': ['n59151784'], 'n15117887': ['n59151785'], 'n15117402': ['n59151786'], 'n15117886': ['n59151787'], 'n15117892': ['n59151788'], 'n15117891': ['n59151789'], 'n15117410': ['n59151790'], 'n15117894': ['n59151791'], 'n15117893': ['n59151792'], 'n15117890': ['n59151793'], 'n15104588': ['n59151794'], 'n15105435': ['n59151795'], 'n15101079': ['n59151796'], 'n15108963': ['n59151797'], 'n15117438': ['n59151798'], 'n15117437': ['n59151799'], 'n15117439': ['n59151800'], 'n15117434': ['n59151801'], 'n15117433': ['n59151802'], 'n15117436': ['n59151803'], 'n15117435': ['n59151804'], 'n15117441': ['n59151805'], 'n15117440': ['n59151806'], 'n15117443': ['n59151807'], 'n15117442': ['n59151808'], 'n15106311': ['n59151809'], 'n15106310': ['n59151810'], 'n15117427': ['n59151811'], 'n15113068': ['n59151812'], 'n15117426': ['n59151813'], 'n15117429': ['n59151814'], 'n15117428': ['n59151815'], 'n15117423': ['n59151816'], 'n15117422': ['n59151817'], 'n15117425': ['n59151818'], 'n15117424': ['n59151819'], 'n15117430': ['n59151820'], 'n15113071': ['n59151821'], 'n15117432': ['n59151822'], 'n15117431': ['n59151823'], 'n15104128': ['n59151824'], 'n15106785': ['n59151825'], 'n15106322': ['n59151826'], 'n15107651': ['n59151827'], 'n15103296': ['n59151828'], 'n15104140': ['n59151829'], 'n15117459': ['n59151830'], 'n15117456': ['n59151831'], 'n15117455': ['n59151832'], 'n15117458': ['n59151833'], 'n15117457': ['n59151834'], 'n15117463': ['n59151835'], 'n15117462': ['n59151836'], 'n15117465': ['n59151837'], 'n15117464': ['n59151838'], 'n15116130': ['n59151839'], 'n15117461': ['n59151840'], 'n15117460': ['n59151841'], 'n15110827': ['n59151842'], 'n15104134': ['n59151843'], 'n15117449': ['n59151844'], 'n15117448': ['n59151845'], 'n15117445': ['n59151846'], 'n15117444': ['n59151847'], 'n15117447': ['n59151848'], 'n15116599': ['n59151849'], 'n15117446': ['n59151850'], 'n15117452': ['n59151851'], 'n15117451': ['n59151852'], 'n15117454': ['n59151853'], 'n15117453': ['n59151854'], 'n15117450': ['n59151855'], 'n15117478': ['n59151856'], 'n15117477': ['n59151857'], 'n15117479': ['n59151858'], 'n15117001': ['n59151859'], 'n15117485': ['n59151860'], 'n15117000': ['n59151861'], 'n15117484': ['n59151862'], 'n15117003': ['n59151863'], 'n15117487': ['n59151864'], 'n15117002': ['n59151865'], 'n15117486': ['n59151866'], 'n15117481': ['n59151867'], 'n15117480': ['n59151868'], 'n15117483': ['n59151869'], 'n15117482': ['n59151870'], 'n15109849': ['n59151871'], 'n15117467': ['n59151872'], 'n15117466': ['n59151873'], 'n15117469': ['n59151874'], 'n15117468': ['n59151875'], 'n15117474': ['n59151876'], 'n15117473': ['n59151877'], 'n15117476': ['n59151878'], 'n15117475': ['n59151879'], 'n15117470': ['n59151880'], 'n15117472': ['n59151881'], 'n15117471': ['n59151882'], 'n15105272': ['n59151883'], 'n15118108': ['n59151884'], 'n15118109': ['n59151885'], 'n15117258': ['n59151886'], 'n15118106': ['n59151887'], 'n15117257': ['n59151888'], 'n15118107': ['n59151889'], 'n15118104': ['n59151890'], 'n15117259': ['n59151891'], 'n15118105': ['n59151892'], 'n15117265': ['n59151893'], 'n15118113': ['n59151894'], 'n15117264': ['n59151895'], 'n15118114': ['n59151896'], 'n15117267': ['n59151897'], 'n15118111': ['n59151898'], 'n15117266': ['n59151899'], 'n15118112': ['n59151900'], 'n15117261': ['n59151901'], 'n15117260': ['n59151902'], 'n15118110': ['n59151903'], 'n15117263': ['n59151904'], 'n15117262': ['n59151905'], 'n15106114': ['n59151906'], 'n15109627': ['n59151907'], 'n15111948': ['n59151908'], 'n15117247': ['n59151909'], 'n15117246': ['n59151910'], 'n15117249': ['n59151911'], 'n15117248': ['n59151912'], 'n15117254': ['n59151913'], 'n15118102': ['n59151914'], 'n15117253': ['n59151915'], 'n15118103': ['n59151916'], 'n15117256': ['n59151917'], 'n15118100': ['n59151918'], 'n15117255': ['n59151919'], 'n15118101': ['n59151920'], 'n15117250': ['n59151921'], 'n15117252': ['n59151922'], 'n15117251': ['n59151923'], 'n15106126': ['n59151924'], 'n15118128': ['n59151925'], 'n15117279': ['n59151926'], 'n15118129': ['n59151927'], 'n15118126': ['n59151928'], 'n15118127': ['n59151929'], 'n15117287': ['n59151930'], 'n15118135': ['n59151931'], 'n15117286': ['n59151932'], 'n15118136': ['n59151933'], 'n15117289': ['n59151934'], 'n15118133': ['n59151935'], 'n15117288': ['n59151936'], 'n15118134': ['n59151937'], 'n15117283': ['n59151938'], 'n15118131': ['n59151939'], 'n15117282': ['n59151940'], 'n15118132': ['n59151941'], 'n15117285': ['n59151942'], 'n15117284': ['n59151943'], 'n15118130': ['n59151944'], 'n15117281': ['n59151945'], 'n15117280': ['n59151946'], 'n15106154': ['n59151947'], 'n15107484': ['n59151948'], 'n15118119': ['n59151949'], 'n15117269': ['n59151950'], 'n15118117': ['n59151951'], 'n15117268': ['n59151952'], 'n15118118': ['n59151953'], 'n15118115': ['n59151954'], 'n15118116': ['n59151955'], 'n15117276': ['n59151956'], 'n15118124': ['n59151957'], 'n15117275': ['n59151958'], 'n15118125': ['n59151959'], 'n15117278': ['n59151960'], 'n15118122': ['n59151961'], 'n15117277': ['n59151962'], 'n15118123': ['n59151963'], 'n15117272': ['n59151964'], 'n15118120': ['n59151965'], 'n15117271': ['n59151966'], 'n15118121': ['n59151967'], 'n15117274': ['n59151968'], 'n15100845': ['n59151969'], 'n15117273': ['n59151970'], 'n15117270': ['n59151971'], 'n15118148': ['n59151972'], 'n15118149': ['n59151973'], 'n15101704': ['n59151974'], 'n15118157': ['n59151975'], 'n15118158': ['n59151976'], 'n15118155': ['n59151977'], 'n15118156': ['n59151978'], 'n15118153': ['n59151979'], 'n15118154': ['n59151980'], 'n15118151': ['n59151981'], 'n15118152': ['n59151982'], 'n15118150': ['n59151983'], 'n15109688': ['n59151984'], 'n15112834': ['n59151985'], 'n15118139': ['n59151986'], 'n15118137': ['n59151987'], 'n15118138': ['n59151988'], 'n15117298': ['n59151989'], 'n15118146': ['n59151990'], 'n15117297': ['n59151991'], 'n15118147': ['n59151992'], 'n15101717': ['n59151993'], 'n15118144': ['n59151994'], 'n15117299': ['n59151995'], 'n15118145': ['n59151996'], 'n15117294': ['n59151997'], 'n15118142': ['n59151998'], 'n15117293': ['n59151999'], 'n15118143': ['n59152000'], 'n15117296': ['n59152001'], 'n15118140': ['n59152002'], 'n15117295': ['n59152003'], 'n15118141': ['n59152004'], 'n15117290': ['n59152005'], 'n15117292': ['n59152006'], 'n15117291': ['n59152007'], 'n15113712': ['n59152008'], 'n15106181': ['n59152009'], 'n15118179': ['n59152010'], 'n15118177': ['n59152011'], 'n15118178': ['n59152012'], 'n15118175': ['n59152013'], 'n15103900': ['n59152014'], 'n15118176': ['n59152015'], 'n15118173': ['n59152016'], 'n15118174': ['n59152017'], 'n15118171': ['n59152018'], 'n15118172': ['n59152019'], 'n15118170': ['n59152020'], 'n15106194': ['n59152021'], 'n15113705': ['n59152022'], 'n15118159': ['n59152023'], 'n15118168': ['n59152024'], 'n15118169': ['n59152025'], 'n15118166': ['n59152026'], 'n15118167': ['n59152027'], 'n15118164': ['n59152028'], 'n15118165': ['n59152029'], 'n15118162': ['n59152030'], 'n15101736': ['n59152031'], 'n15118163': ['n59152032'], 'n15118160': ['n59152033'], 'n15118161': ['n59152034'], 'n15108381': ['n59152035'], 'n15118199': ['n59152036'], 'n15118197': ['n59152037'], 'n15118198': ['n59152038'], 'n15118195': ['n59152039'], 'n15118196': ['n59152040'], 'n15118193': ['n59152041'], 'n15118194': ['n59152042'], 'n15118191': ['n59152043'], 'n15118192': ['n59152044'], 'n15118190': ['n59152045'], 'n15111542': ['n59152046'], 'n15118188': ['n59152047'], 'n15118189': ['n59152048'], 'n15118186': ['n59152049'], 'n15118187': ['n59152050'], 'n15118184': ['n59152051'], 'n15118185': ['n59152052'], 'n15118182': ['n59152053'], 'n15118183': ['n59152054'], 'n15118180': ['n59152055'], 'n15118181': ['n59152056'], 'n15110248': ['n59152057'], 'n15100433': ['n59152058'], 'n15113747': ['n59152059'], 'n15101775': ['n59152060'], 'n15113779': ['n59152061'], 'n15115956': ['n59152062'], 'n15107093': ['n59152063'], 'n15110265': ['n59152064'], 'n15101308': ['n59152065'], 'n15104817': ['n59152066'], 'n15101305': ['n59152067'], 'n15101786': ['n59152068'], 'n15115941': ['n59152069'], 'n15102644': ['n59152070'], 'n15109279': ['n59152071'], 'n15101793': ['n59152072'], 'n15113792': ['n59152073'], 'n15110289': ['n59152074'], 'n15114640': ['n59152075'], 'n15103504': ['n59152076'], 'n15116812': ['n59152077'], 'n15102665': ['n59152078'], 'n15101331': ['n59152079'], 'n15102672': ['n59152080'], 'n15100494': ['n59152081'], 'n15100012': ['n59152082'], 'n15110295': ['n59152083'], 'n15114657': ['n59152084'], 'n15111147': ['n59152085'], 'n15114650': ['n59152086'], 'n15103534': ['n59152087'], 'n15100027': ['n59152088'], 'n15101372': ['n59152089'], 'n15117717': ['n59152090'], 'n15117716': ['n59152091'], 'n15117719': ['n59152092'], 'n15117718': ['n59152093'], 'n15117713': ['n59152094'], 'n15117712': ['n59152095'], 'n15117715': ['n59152096'], 'n15117714': ['n59152097'], 'n15117711': ['n59152098'], 'n15117710': ['n59152099'], 'n15107907': ['n59152100'], 'n15102693': ['n59152101'], 'n15105721': ['n59152102'], 'n15117709': ['n59152103'], 'n15117706': ['n59152104'], 'n15117705': ['n59152105'], 'n15117708': ['n59152106'], 'n15117707': ['n59152107'], 'n15117702': ['n59152108'], 'n15117701': ['n59152109'], 'n15117704': ['n59152110'], 'n15117703': ['n59152111'], 'n15117700': ['n59152112'], 'n15105737': ['n59152113'], 'n15107913': ['n59152114'], 'n15100048': ['n59152115'], 'n15100049': ['n59152116'], 'n15117739': ['n59152117'], 'n15117738': ['n59152118'], 'n15117735': ['n59152119'], 'n15117734': ['n59152120'], 'n15117737': ['n59152121'], 'n15111197': ['n59152122'], 'n15117736': ['n59152123'], 'n15117731': ['n59152124'], 'n15117730': ['n59152125'], 'n15117733': ['n59152126'], 'n15117732': ['n59152127'], 'n15117740': ['n59152128'], 'n15117728': ['n59152129'], 'n15117727': ['n59152130'], 'n15117729': ['n59152131'], 'n15117724': ['n59152132'], 'n15117723': ['n59152133'], 'n15117726': ['n59152134'], 'n15117725': ['n59152135'], 'n15117720': ['n59152136'], 'n15117722': ['n59152137'], 'n15117721': ['n59152138'], 'n15106607': ['n59152139'], 'n15106606': ['n59152140'], 'n15107934': ['n59152141'], 'n15117757': ['n59152142'], 'n15117756': ['n59152143'], 'n15117759': ['n59152144'], 'n15117758': ['n59152145'], 'n15117753': ['n59152146'], 'n15117752': ['n59152147'], 'n15117755': ['n59152148'], 'n15116423': ['n59152149'], 'n15117754': ['n59152150'], 'n15117760': ['n59152151'], 'n15117762': ['n59152152'], 'n15117761': ['n59152153'], 'n15117749': ['n59152154'], 'n15117746': ['n59152155'], 'n15117745': ['n59152156'], 'n15117748': ['n59152157'], 'n15117747': ['n59152158'], 'n15117742': ['n59152159'], 'n15117741': ['n59152160'], 'n15117744': ['n59152161'], 'n15117743': ['n59152162'], 'n15117751': ['n59152163'], 'n15117750': ['n59152164'], 'n15106641': ['n59152165'], 'n15105791': ['n59152166'], 'n15117779': ['n59152167'], 'n15117778': ['n59152168'], 'n15117775': ['n59152169'], 'n15117774': ['n59152170'], 'n15117777': ['n59152171'], 'n15117776': ['n59152172'], 'n15117782': ['n59152173'], 'n15117781': ['n59152174'], 'n15117300': ['n59152175'], 'n15117784': ['n59152176'], 'n15112090': ['n59152177'], 'n15116452': ['n59152178'], 'n15117783': ['n59152179'], 'n15117780': ['n59152180'], 'n15107969': ['n59152181'], 'n15103124': ['n59152182'], 'n15107980': ['n59152183'], 'n15117768': ['n59152184'], 'n15117767': ['n59152185'], 'n15116439': ['n59152186'], 'n15117769': ['n59152187'], 'n15117764': ['n59152188'], 'n15117763': ['n59152189'], 'n15117766': ['n59152190'], 'n15117765': ['n59152191'], 'n15117771': ['n59152192'], 'n15117770': ['n59152193'], 'n15117773': ['n59152194'], 'n15117772': ['n59152195'], 'n15106644': ['n59152196'], 'n15106660': ['n59152197'], 'n15117317': ['n59152198'], 'n15117316': ['n59152199'], 'n15117319': ['n59152200'], 'n15117318': ['n59152201'], 'n15117313': ['n59152202'], 'n15117797': ['n59152203'], 'n15117312': ['n59152204'], 'n15117796': ['n59152205'], 'n15117315': ['n59152206'], 'n15117799': ['n59152207'], 'n15117314': ['n59152208'], 'n15117798': ['n59152209'], 'n15117320': ['n59152210'], 'n15117322': ['n59152211'], 'n15117321': ['n59152212'], 'n15107507': ['n59152213'], 'n15117309': ['n59152214'], 'n15117306': ['n59152215'], 'n15117305': ['n59152216'], 'n15117789': ['n59152217'], 'n15115129': ['n59152218'], 'n15117308': ['n59152219'], 'n15117307': ['n59152220'], 'n15117302': ['n59152221'], 'n15117786': ['n59152222'], 'n15117301': ['n59152223'], 'n15117785': ['n59152224'], 'n15117304': ['n59152225'], 'n15117788': ['n59152226'], 'n15117303': ['n59152227'], 'n15117787': ['n59152228'], 'n15117793': ['n59152229'], 'n15117792': ['n59152230'], 'n15117311': ['n59152231'], 'n15117795': ['n59152232'], 'n15117310': ['n59152233'], 'n15117794': ['n59152234'], 'n15117791': ['n59152235'], 'n15117790': ['n59152236'], 'n15108841': ['n59152237'], 'n15117339': ['n59152238'], 'n15117338': ['n59152239'], 'n15117335': ['n59152240'], 'n15117334': ['n59152241'], 'n15117337': ['n59152242'], 'n15117336': ['n59152243'], 'n15117342': ['n59152244'], 'n15117341': ['n59152245'], 'n15117344': ['n59152246'], 'n15117343': ['n59152247'], 'n15117340': ['n59152248'], 'n15108854': ['n59152249'], 'n15117328': ['n59152250'], 'n15117327': ['n59152251'], 'n15117329': ['n59152252'], 'n15117324': ['n59152253'], 'n15117323': ['n59152254'], 'n15117326': ['n59152255'], 'n15117325': ['n59152256'], 'n15117331': ['n59152257'], 'n15117330': ['n59152258'], 'n15117333': ['n59152259'], 'n15117332': ['n59152260'], 'n15103195': ['n59152261'], 'n15109734': ['n59152262'], 'n15112900': ['n59152263'], 'n15110725': ['n59152264'], 'n15118209': ['n59152265'], 'n15118207': ['n59152266'], 'n15118208': ['n59152267'], 'n15117357': ['n59152268'], 'n15118205': ['n59152269'], 'n15117356': ['n59152270'], 'n15118206': ['n59152271'], 'n15117359': ['n59152272'], 'n15118203': ['n59152273'], 'n15117358': ['n59152274'], 'n15118204': ['n59152275'], 'n15117364': ['n59152276'], 'n15118212': ['n59152277'], 'n15117363': ['n59152278'], 'n15118213': ['n59152279'], 'n15117366': ['n59152280'], 'n15118210': ['n59152281'], 'n15117365': ['n59152282'], 'n15118211': ['n59152283'], 'n15117360': ['n59152284'], 'n15117362': ['n59152285'], 'n15117361': ['n59152286'], 'n15109743': ['n59152287'], 'n15107564': ['n59152288'], 'n15108876': ['n59152289'], 'n15110713': ['n59152290'], 'n15117349': ['n59152291'], 'n15117346': ['n59152292'], 'n15117345': ['n59152293'], 'n15117348': ['n59152294'], 'n15117347': ['n59152295'], 'n15117353': ['n59152296'], 'n15118201': ['n59152297'], 'n15117352': ['n59152298'], 'n15118202': ['n59152299'], 'n15117355': ['n59152300'], 'n15117354': ['n59152301'], 'n15118200': ['n59152302'], 'n15100921': ['n59152303'], 'n15117351': ['n59152304'], 'n15117350': ['n59152305'], 'n15117137': ['n59152306'], 'n15117136': ['n59152307'], 'n15117139': ['n59152308'], 'n15117138': ['n59152309'], 'n15117144': ['n59152310'], 'n15117143': ['n59152311'], 'n15117146': ['n59152312'], 'n15117145': ['n59152313'], 'n15117140': ['n59152314'], 'n15117142': ['n59152315'], 'n15117141': ['n59152316'], 'n15109506': ['n59152317'], 'n15117129': ['n59152318'], 'n15117126': ['n59152319'], 'n15117125': ['n59152320'], 'n15117128': ['n59152321'], 'n15117127': ['n59152322'], 'n15117133': ['n59152323'], 'n15117132': ['n59152324'], 'n15117135': ['n59152325'], 'n15117134': ['n59152326'], 'n15117131': ['n59152327'], 'n15116283': ['n59152328'], 'n15117130': ['n59152329'], 'n15107339': ['n59152330'], 'n15118009': ['n59152331'], 'n15117159': ['n59152332'], 'n15118007': ['n59152333'], 'n15117158': ['n59152334'], 'n15118008': ['n59152335'], 'n15118005': ['n59152336'], 'n15118006': ['n59152337'], 'n15117166': ['n59152338'], 'n15118014': ['n59152339'], 'n15117165': ['n59152340'], 'n15118015': ['n59152341'], 'n15117168': ['n59152342'], 'n15118012': ['n59152343'], 'n15117167': ['n59152344'], 'n15118013': ['n59152345'], 'n15117162': ['n59152346'], 'n15118010': ['n59152347'], 'n15117161': ['n59152348'], 'n15118011': ['n59152349'], 'n15117164': ['n59152350'], 'n15100713': ['n59152351'], 'n15117163': ['n59152352'], 'n15117160': ['n59152353'], 'n15108214': ['n59152354'], 'n15108213': ['n59152355'], 'n15117148': ['n59152356'], 'n15117147': ['n59152357'], 'n15117149': ['n59152358'], 'n15117155': ['n59152359'], 'n15118003': ['n59152360'], 'n15117154': ['n59152361'], 'n15118004': ['n59152362'], 'n15117157': ['n59152363'], 'n15118001': ['n59152364'], 'n15117156': ['n59152365'], 'n15118002': ['n59152366'], 'n15117151': ['n59152367'], 'n15117150': ['n59152368'], 'n15118000': ['n59152369'], 'n15117153': ['n59152370'], 'n15117152': ['n59152371'], 'n15106027': ['n59152372'], 'n15105196': ['n59152373'], 'n15118029': ['n59152374'], 'n15111871': ['n59152375'], 'n15118027': ['n59152376'], 'n15111872': ['n59152377'], 'n15118028': ['n59152378'], 'n15117188': ['n59152379'], 'n15118036': ['n59152380'], 'n15117187': ['n59152381'], 'n15118037': ['n59152382'], 'n15118034': ['n59152383'], 'n15117189': ['n59152384'], 'n15118035': ['n59152385'], 'n15117184': ['n59152386'], 'n15118032': ['n59152387'], 'n15117183': ['n59152388'], 'n15118033': ['n59152389'], 'n15117186': ['n59152390'], 'n15118030': ['n59152391'], 'n15117185': ['n59152392'], 'n15118031': ['n59152393'], 'n15117180': ['n59152394'], 'n15117182': ['n59152395'], 'n15117181': ['n59152396'], 'n15109547': ['n59152397'], 'n15106053': ['n59152398'], 'n15118018': ['n59152399'], 'n15110532': ['n59152400'], 'n15117169': ['n59152401'], 'n15118019': ['n59152402'], 'n15118016': ['n59152403'], 'n15118017': ['n59152404'], 'n15117177': ['n59152405'], 'n15118025': ['n59152406'], 'n15117176': ['n59152407'], 'n15118026': ['n59152408'], 'n15117179': ['n59152409'], 'n15118023': ['n59152410'], 'n15117178': ['n59152411'], 'n15118024': ['n59152412'], 'n15117173': ['n59152413'], 'n15118021': ['n59152414'], 'n15117172': ['n59152415'], 'n15118022': ['n59152416'], 'n15117175': ['n59152417'], 'n15117174': ['n59152418'], 'n15118020': ['n59152419'], 'n15117171': ['n59152420'], 'n15100742': ['n59152421'], 'n15117170': ['n59152422'], 'n15118049': ['n59152423'], 'n15111894': ['n59152424'], 'n15118058': ['n59152425'], 'n15100759': ['n59152426'], 'n15118059': ['n59152427'], 'n15118056': ['n59152428'], 'n15118057': ['n59152429'], 'n15118054': ['n59152430'], 'n15118055': ['n59152431'], 'n15118052': ['n59152432'], 'n15118053': ['n59152433'], 'n15118050': ['n59152434'], 'n15118051': ['n59152435'], 'n15106078': ['n59152436'], 'n15106077': ['n59152437'], 'n15118038': ['n59152438'], 'n15118039': ['n59152439'], 'n15117199': ['n59152440'], 'n15118047': ['n59152441'], 'n15117198': ['n59152442'], 'n15118048': ['n59152443'], 'n15118045': ['n59152444'], 'n15118046': ['n59152445'], 'n15117195': ['n59152446'], 'n15118043': ['n59152447'], 'n15117194': ['n59152448'], 'n15118044': ['n59152449'], 'n15117197': ['n59152450'], 'n15118041': ['n59152451'], 'n15117196': ['n59152452'], 'n15118042': ['n59152453'], 'n15117191': ['n59152454'], 'n15117190': ['n59152455'], 'n15118040': ['n59152456'], 'n15117193': ['n59152457'], 'n15117192': ['n59152458'], 'n15100760': ['n59152459'], 'n15118078': ['n59152460'], 'n15118079': ['n59152461'], 'n15118076': ['n59152462'], 'n15118077': ['n59152463'], 'n15118074': ['n59152464'], 'n15118075': ['n59152465'], 'n15118072': ['n59152466'], 'n15118073': ['n59152467'], 'n15118070': ['n59152468'], 'n15118071': ['n59152469'], 'n15109127': ['n59152470'], 'n15108277': ['n59152471'], 'n15106094': ['n59152472'], 'n15111422': ['n59152473'], 'n15112754': ['n59152474'], 'n15118069': ['n59152475'], 'n15118067': ['n59152476'], 'n15118068': ['n59152477'], 'n15118065': ['n59152478'], 'n15118066': ['n59152479'], 'n15103815': ['n59152480'], 'n15118063': ['n59152481'], 'n15118064': ['n59152482'], 'n15118061': ['n59152483'], 'n15118062': ['n59152484'], 'n15118060': ['n59152485'], 'n15109597': ['n59152486'], 'n15109114': ['n59152487'], 'n15112780': ['n59152488'], 'n15118098': ['n59152489'], 'n15118099': ['n59152490'], 'n15109140': ['n59152491'], 'n15118096': ['n59152492'], 'n15118097': ['n59152493'], 'n15118094': ['n59152494'], 'n15118095': ['n59152495'], 'n15118092': ['n59152496'], 'n15118093': ['n59152497'], 'n15118090': ['n59152498'], 'n15118091': ['n59152499'], 'n15100329': ['n59152500'], 'n15118089': ['n59152501'], 'n15118087': ['n59152502'], 'n15118088': ['n59152503'], 'n15118085': ['n59152504'], 'n15118086': ['n59152505'], 'n15118083': ['n59152506'], 'n15118084': ['n59152507'], 'n15118081': ['n59152508'], 'n15118082': ['n59152509'], 'n15118080': ['n59152510'], 'n15109160': ['n59152511'], 'n15101666': ['n59152512'], 'n15112313': ['n59152513'], 'n15103853': ['n59152514'], 'n15112350': ['n59152515'], 'n15109183': ['n59152516'], 'n15101202': ['n59152517'], 'n15111484': ['n59152518'], 'n15112331': ['n59152519'], 'n15110152': ['n59152520'], 'n15114993': ['n59152521'], 'n15115879': ['n59152522'], 'n15104730': ['n59152523'], 'n15115868': ['n59152524'], 'n15116711': ['n59152525'], 'n15114530': ['n59152526'], 'n15109196': ['n59152527'], 'n15114562': ['n59152528'], 'n15101244': ['n59152529'], 'n15105614': ['n59152530'], 'n15104764': ['n59152531'], 'n15117618': ['n59152532'], 'n15117617': ['n59152533'], 'n15117619': ['n59152534'], 'n15117614': ['n59152535'], 'n15111078': ['n59152536'], 'n15117613': ['n59152537'], 'n15117616': ['n59152538'], 'n15117615': ['n59152539'], 'n15117610': ['n59152540'], 'n15117612': ['n59152541'], 'n15117611': ['n59152542'], 'n15115440': ['n59152543'], 'n15114594': ['n59152544'], 'n15102598': ['n59152545'], 'n15104772': ['n59152546'], 'n15117607': ['n59152547'], 'n15117606': ['n59152548'], 'n15117609': ['n59152549'], 'n15117608': ['n59152550'], 'n15117603': ['n59152551'], 'n15117602': ['n59152552'], 'n15117605': ['n59152553'], 'n15116757': ['n59152554'], 'n15117604': ['n59152555'], 'n15117601': ['n59152556'], 'n15117600': ['n59152557'], 'n15104787': ['n59152558'], 'n15102124': ['n59152559'], 'n15117639': ['n59152560'], 'n15117636': ['n59152561'], 'n15117635': ['n59152562'], 'n15117638': ['n59152563'], 'n15117637': ['n59152564'], 'n15117632': ['n59152565'], 'n15117631': ['n59152566'], 'n15117634': ['n59152567'], 'n15114125': ['n59152568'], 'n15117633': ['n59152569'], 'n15117641': ['n59152570'], 'n15114132': ['n59152571'], 'n15115463': ['n59152572'], 'n15117640': ['n59152573'], 'n15104316': ['n59152574'], 'n15106975': ['n59152575'], 'n15104794': ['n59152576'], 'n15107840': ['n59152577'], 'n15102150': ['n59152578'], 'n15117629': ['n59152579'], 'n15117628': ['n59152580'], 'n15117625': ['n59152581'], 'n15117624': ['n59152582'], 'n15117627': ['n59152583'], 'n15117626': ['n59152584'], 'n15117621': ['n59152585'], 'n15117620': ['n59152586'], 'n15117623': ['n59152587'], 'n15114598': ['n59152588'], 'n15117622': ['n59152589'], 'n15115450': ['n59152590'], 'n15117630': ['n59152591'], 'n15117658': ['n59152592'], 'n15117657': ['n59152593'], 'n15117659': ['n59152594'], 'n15117654': ['n59152595'], 'n15117653': ['n59152596'], 'n15117656': ['n59152597'], 'n15117655': ['n59152598'], 'n15117661': ['n59152599'], 'n15117660': ['n59152600'], 'n15117663': ['n59152601'], 'n15117662': ['n59152602'], 'n15103021': ['n59152603'], 'n15117647': ['n59152604'], 'n15117646': ['n59152605'], 'n15117649': ['n59152606'], 'n15117648': ['n59152607'], 'n15117643': ['n59152608'], 'n15117642': ['n59152609'], 'n15117645': ['n59152610'], 'n15117644': ['n59152611'], 'n15117650': ['n59152612'], 'n15117652': ['n59152613'], 'n15117651': ['n59152614'], 'n15104349': ['n59152615'], 'n15106523': ['n59152616'], 'n15103499': ['n59152617'], 'n15105690': ['n59152618'], 'n15117679': ['n59152619'], 'n15117676': ['n59152620'], 'n15117675': ['n59152621'], 'n15117678': ['n59152622'], 'n15117677': ['n59152623'], 'n15117683': ['n59152624'], 'n15117682': ['n59152625'], 'n15117201': ['n59152626'], 'n15117685': ['n59152627'], 'n15117200': ['n59152628'], 'n15117684': ['n59152629'], 'n15117681': ['n59152630'], 'n15117680': ['n59152631'], 'n15102176': ['n59152632'], 'n15104357': ['n59152633'], 'n15117669': ['n59152634'], 'n15117668': ['n59152635'], 'n15117665': ['n59152636'], 'n15117664': ['n59152637'], 'n15117667': ['n59152638'], 'n15117666': ['n59152639'], 'n15117672': ['n59152640'], 'n15117671': ['n59152641'], 'n15117674': ['n59152642'], 'n15117673': ['n59152643'], 'n15117670': ['n59152644'], 'n15107875': ['n59152645'], 'n15108747': ['n59152646'], 'n15117218': ['n59152647'], 'n15117217': ['n59152648'], 'n15117219': ['n59152649'], 'n15117214': ['n59152650'], 'n15117698': ['n59152651'], 'n15117213': ['n59152652'], 'n15117697': ['n59152653'], 'n15117216': ['n59152654'], 'n15117215': ['n59152655'], 'n15117699': ['n59152656'], 'n15117221': ['n59152657'], 'n15117220': ['n59152658'], 'n15117223': ['n59152659'], 'n15117222': ['n59152660'], 'n15117207': ['n59152661'], 'n15117206': ['n59152662'], 'n15117209': ['n59152663'], 'n15117208': ['n59152664'], 'n15117203': ['n59152665'], 'n15117687': ['n59152666'], 'n15117202': ['n59152667'], 'n15117686': ['n59152668'], 'n15117205': ['n59152669'], 'n15117689': ['n59152670'], 'n15117204': ['n59152671'], 'n15117688': ['n59152672'], 'n15117210': ['n59152673'], 'n15117694': ['n59152674'], 'n15117693': ['n59152675'], 'n15117212': ['n59152676'], 'n15117696': ['n59152677'], 'n15117211': ['n59152678'], 'n15117695': ['n59152679'], 'n15117690': ['n59152680'], 'n15117692': ['n59152681'], 'n15117691': ['n59152682'], 'n15111906': ['n59152683'], 'n15105236': ['n59152684'], 'n15117239': ['n59152685'], 'n15117236': ['n59152686'], 'n15117235': ['n59152687'], 'n15117238': ['n59152688'], 'n15117237': ['n59152689'], 'n15117243': ['n59152690'], 'n15117242': ['n59152691'], 'n15117245': ['n59152692'], 'n15117244': ['n59152693'], 'n15117241': ['n59152694'], 'n15117240': ['n59152695'], 'n15108774': ['n59152696'], 'n15108756': ['n59152697'], 'n15106594': ['n59152698'], 'n15111920': ['n59152699'], 'n15117229': ['n59152700'], 'n15117228': ['n59152701'], 'n15117225': ['n59152702'], 'n15117224': ['n59152703'], 'n15117227': ['n59152704'], 'n15117226': ['n59152705'], 'n15117232': ['n59152706'], 'n15117231': ['n59152707'], 'n15117234': ['n59152708'], 'n15117233': ['n59152709'], 'n15117230': ['n59152710'], 'n15108760': ['n59152711']}

In [13]:
print(len(edges))

178155040


In [16]:
nodes_types = [nodes[target]['type'] for source, target, edge_attr in edges if edge_attr['label'] == "addedValues"]

In [17]:
from collections import Counter

print(Counter(nodes_types))

Counter({'CVE': 14459139, 'FRESHNESS': 14459139, 'POPULARITY_1_YEAR': 14459139, 'SPEED': 658078})


In [23]:
from collections import defaultdict

addvalue_dict = defaultdict(list)

# Iterate over the edges and add the targets for each source where the label is 'addedValues'
for source, target, edge_att in edges:
    if edge_att['label'] == "addedValues":
        addvalue_dict[source].append(target)


In [24]:
node_type = []
for node_id, node_list in addvalue_dict.items():
    for node in node_list:
        node_type.append(nodes[node]['type'])

In [25]:
from collections import Counter

print(Counter(node_type))

Counter({'CVE': 14459139, 'FRESHNESS': 14459139, 'POPULARITY_1_YEAR': 14459139, 'SPEED': 658078})


In [10]:
print(nodes['n59150222'])
print(nodes['n59150253'])
print(nodes['n59150323'])
print(nodes['n59150369'])
print(nodes['n59150454'])
print(nodes['n59150455'])
print(nodes['n59150463'])
print(nodes['n14990848'])

{'labels': ':AddedValue', 'id': 'io.github.rzo1.org.apache.cxf:cxf-rt-transports-udp:SPEED', 'type': 'SPEED', 'value': '0.0'}
{'labels': ':AddedValue', 'id': 'dev.alteration.x10:ursula_3:SPEED', 'type': 'SPEED', 'value': '0.0'}
{'labels': ':AddedValue', 'type': 'SPEED', 'value': '0.0', 'id': 'com.jardad.lib:jrandom:SPEED'}
{'labels': ':AddedValue', 'id': 'net.sf.jasperreports:jasperreports-spring:SPEED', 'type': 'SPEED', 'value': '0.0'}
{'labels': ':AddedValue', 'type': 'SPEED', 'value': '0.12295081967213115', 'id': 'io.github.lyxnx:util:SPEED'}
{'labels': ':AddedValue', 'id': 'org.specs2:specs2-fp_native0.5_3:SPEED', 'type': 'SPEED', 'value': '0.07407407407407407'}
{'labels': ':AddedValue', 'id': 'io.github.aliyun-sls:alibabacloud-android-rum-plugin:SPEED', 'type': 'SPEED', 'value': '0.0'}
{'labels': ':Artifact', 'found': 'true', 'id': 'org.scala-native:scala-native-java-logging_native0.5_2.13'}


In [33]:
# Extract the labels from each node
node_labels = [node['labels'] for node in nodes.values()]

In [34]:
from collections import Counter
print("Node Label Count is:", Counter(node_labels))

Node Label Count is: Counter({':AddedValue': 44035495, ':Release': 14459139, ':Artifact': 658078})


## check the attributes of edges

In [5]:
edges[:10]

[('n2', 'n13', {'label': 'relationship_AR'}),
 ('n1', 'n14', {'label': 'relationship_AR'}),
 ('n1', 'n7', {'label': 'relationship_AR'}),
 ('n1', 'n8', {'label': 'relationship_AR'}),
 ('n2', 'n9', {'label': 'relationship_AR'}),
 ('n2', 'n4', {'label': 'relationship_AR'}),
 ('n1', 'n11', {'label': 'relationship_AR'}),
 ('n1', 'n3', {'label': 'relationship_AR'}),
 ('n1', 'n6', {'label': 'relationship_AR'}),
 ('n2', 'n10', {'label': 'relationship_AR'})]

In [40]:
from collections import defaultdict

addvalue_dict = defaultdict(list)

# Iterate over the edges and add the targets for each source where the label is 'addedValues'
for source, target, edge_att in edges:
    if edge_att['label'] == "addedValues":
        addvalue_dict[source].append(target)

In [ ]:
addvalue_nodes = {}

In [21]:
nodes['n14078286']

{'labels': ':AddedValue',
 'value': '{\\"cve\\":[]}',
 'id': 'com.softwaremill.sttp.client:core_sjs0.6_2.13:2.0.2:CVE',
 'type': 'CVE'}

In [29]:
def str_to_json(escaped_json_str):
    try:
        clean_str = escaped_json_str.replace('\\"', '"')
        return json.loads(clean_str)
    except ValueError as e:
        print(f"Error parsing JSON: {e}")
        return None

def get_addvalue_edges(edges):
    # source node is release, target node is addedvalue
    return {source: target for source, target, edge_att in edges if edge_att['label'] == "addedValues"}

def cve_check(target:str, nodes, addvalue_edges_dict):
    node = nodes[addvalue_edges_dict[target]]
    if 'type' in node and node['type'] == "CVE" and str_to_json(node["value"])['cve'] !=[]:
        return True
    else:
        return False

In [42]:
node_types = []
for target, node_attr in nodes.items():
    if target in addvalue_dict:
        node_list = addvalue_dict[target]
        for node in node_list:
            node_types.append(nodes[node]['type'])
print(Counter(node_types))

Counter({'CVE': 14459139, 'FRESHNESS': 14459139, 'POPULARITY_1_YEAR': 14459139, 'SPEED': 658078})


In [19]:

print(Counter(edge_labels))

Counter({'dependency': 119660406, 'addedValues': 44035495, 'relationship_AR': 14459139})


In [16]:
len(dep_edges)

0

In [14]:
len(dep_edges)

119660406

In [5]:
nodes_with_timestamp = {node_id: node for node_id, node in nodes.items() if 'timestamp' in node}

In [6]:
for node_id, node in list(nodes_with_timestamp.items())[:10]:
    print(node_id, node)

n3 {'labels': ':Release', 'id': 'org.wso2.carbon.identity.framework:org.wso2.carbon.identity.cors.mgt.core:5.20.111', 'version': '5.20.111', 'timestamp': '1626148242000'}
n4 {'labels': ':Release', 'id': 'org.apache.camel.quarkus:camel-quarkus-kotlin-parent:1.0.0-M4', 'version': '1.0.0-M4', 'timestamp': '1583239943000'}
n5 {'labels': ':Release', 'id': 'org.apache.camel.quarkus:camel-quarkus-kotlin-parent:1.0.0-M3', 'version': '1.0.0-M3', 'timestamp': '1579861029000'}
n6 {'labels': ':Release', 'id': 'org.wso2.carbon.identity.framework:org.wso2.carbon.identity.cors.mgt.core:5.20.113', 'version': '5.20.113', 'timestamp': '1626179580000'}
n7 {'labels': ':Release', 'id': 'org.wso2.carbon.identity.framework:org.wso2.carbon.identity.cors.mgt.core:5.20.112', 'version': '5.20.112', 'timestamp': '1626170945000'}
n8 {'labels': ':Release', 'id': 'org.wso2.carbon.identity.framework:org.wso2.carbon.identity.cors.mgt.core:5.20.115', 'version': '5.20.115', 'timestamp': '1626340086000'}
n9 {'labels': ':

In [6]:
import json

def str_to_json(escaped_json_str):
    try:
        clean_str = escaped_json_str.replace('\\"', '"')
        return json.loads(clean_str)
    except ValueError as e:
        print(f"Error parsing JSON: {e}")
        return None

In [7]:
# Filter nodes that have the 'severity' attribute
nodes_with_severity = {node_id: node for node_id, node in nodes.items() if 'type' in node and node['type'] == "CVE" and str_to_json(node["value"])['cve'] !=[]}
# nodes_with_severity = {node_id: node for node_id, node in nodes.items() if 'type' in node and str_to_json(node["value"])["cve"]!=[]}

In [8]:
print(len({node_id: node for node_id, node in nodes.items() if 'type' in node and node['type'] == "CVE"}))
print(len(nodes_with_severity))

14459139
77393


In [6]:
# Using list slicing to print the first two items
for node_id, node in list(nodes_with_severity.items())[:10]:
    print(node_id, node)

n14293237 {'labels': ':AddedValue', 'id': 'org.keycloak:keycloak-core:3.4.1.Final:CVE', 'type': 'CVE', 'value': '{\\"cve\\":[{\\"cwe\\":\\"[CWE-267]\\",\\"severity\\":\\"HIGH\\",\\"name\\":\\"CVE-2019-10170\\"},{\\"cwe\\":\\"[CWE-79]\\",\\"severity\\":\\"MODERATE\\",\\"name\\":\\"CVE-2022-0225\\"},{\\"cwe\\":\\"[CWE-79]\\",\\"severity\\":\\"MODERATE\\",\\"name\\":\\"CVE-2020-1697\\"},{\\"cwe\\":\\"[CWE-547,CWE-798]\\",\\"severity\\":\\"CRITICAL\\",\\"name\\":\\"CVE-2019-14837\\"},{\\"cwe\\":\\"[CWE-306]\\",\\"severity\\":\\"MODERATE\\",\\"name\\":\\"CVE-2021-20262\\"},{\\"cwe\\":\\"[CWE-1021]\\",\\"severity\\":\\"MODERATE\\",\\"name\\":\\"CVE-2020-1728\\"},{\\"cwe\\":\\"[CWE-285,CWE-287]\\",\\"severity\\":\\"HIGH\\",\\"name\\":\\"CVE-2018-14637\\"},{\\"cwe\\":\\"[CWE-276]\\",\\"severity\\":\\"LOW\\",\\"name\\":\\"UNKNOWN\\"},{\\"cwe\\":\\"[CWE-285]\\",\\"severity\\":\\"MODERATE\\",\\"name\\":\\"CVE-2020-10686\\"},{\\"cwe\\":\\"[CWE-20]\\",\\"severity\\":\\"HIGH\\",\\"name\\":\\"CVE-202

In [8]:
example_node= {"n0": 
               {'labels': ':AddedValue', 
                 'id': 'org.keycloak:keycloak-core:3.4.1.Final:CVE', 'type': 'CVE', 'value': '{\\"cve\\":[{\\"cwe\\":\\"[CWE-267]\\",\\"severity\\":\\"HIGH\\",\\"name\\":\\"CVE-2019-10170\\"},{\\"cwe\\":\\"[CWE-79]\\",\\"severity\\":\\"MODERATE\\",\\"name\\":\\"CVE-2022-0225\\"},{\\"cwe\\":\\"[CWE-79]\\",\\"severity\\":\\"MODERATE\\",\\"name\\":\\"CVE-2020-1697\\"},{\\"cwe\\":\\"[CWE-547,CWE-798]\\",\\"severity\\":\\"CRITICAL\\",\\"name\\":\\"CVE-2019-14837\\"},{\\"cwe\\":\\"[CWE-306]\\",\\"severity\\":\\"MODERATE\\",\\"name\\":\\"CVE-2021-20262\\"},{\\"cwe\\":\\"[CWE-1021]\\",\\"severity\\":\\"MODERATE\\",\\"name\\":\\"CVE-2020-1728\\"},{\\"cwe\\":\\"[CWE-285,CWE-287]\\",\\"severity\\":\\"HIGH\\",\\"name\\":\\"CVE-2018-14637\\"},{\\"cwe\\":\\"[CWE-276]\\",\\"severity\\":\\"LOW\\",\\"name\\":\\"UNKNOWN\\"},{\\"cwe\\":\\"[CWE-285]\\",\\"severity\\":\\"MODERATE\\",\\"name\\":\\"CVE-2020-10686\\"},{\\"cwe\\":\\"[CWE-20]\\",\\"severity\\":\\"HIGH\\",\\"name\\":\\"CVE-2020-1714\\"},{\\"cwe\\":\\"[CWE-287,CWE-841]\\",\\"severity\\":\\"MODERATE\\",\\"name\\":\\"UNKNOWN\\"},{\\"cwe\\":\\"[CWE-613]\\",\\"severity\\":\\"MODERATE\\",\\"name\\":\\"CVE-2020-1724\\"},{\\"cwe\\":\\"[CWE-835]\\",\\"severity\\":\\"MODERATE\\",\\"name\\":\\"CVE-2018-10912\\"},{\\"cwe\\":\\"[CWE-287]\\",\\"severity\\":\\"MODERATE\\",\\"name\\":\\"CVE-2020-27838\\"},{\\"cwe\\":\\"[CWE-287,CWE-841]\\",\\"severity\\":\\"MODERATE\\",\\"name\\":\\"CVE-2023-0105\\"},{\\"cwe\\":\\"[CWE-200,CWE-755]\\",\\"severity\\":\\"MODERATE\\",\\"name\\":\\"CVE-2020-1744\\"},{\\"cwe\\":\\"[CWE-295,CWE-345]\\",\\"severity\\":\\"MODERATE\\",\\"name\\":\\"CVE-2019-3875\\"},{\\"cwe\\":\\"[CWE-601]\\",\\"severity\\":\\"MODERATE\\",\\"name\\":\\"UNKNOWN\\"},{\\"cwe\\":\\"[CWE-200,CWE-532]\\",\\"severity\\":\\"MODERATE\\",\\"name\\":\\"CVE-2020-1698\\"},{\\"cwe\\":\\"[CWE-863]\\",\\"severity\\":\\"MODERATE\\",\\"name\\":\\"CVE-2022-1466\\"},{\\"cwe\\":\\"[CWE-200]\\",\\"severity\\":\\"MODERATE\\",\\"name\\":\\"CVE-2019-14820\\"},{\\"cwe\\":\\"[CWE-295]\\",\\"severity\\":\\"MODERATE\\",\\"name\\":\\"UNKNOWN\\"},{\\"cwe\\":\\"[CWE-250]\\",\\"severity\\":\\"HIGH\\",\\"name\\":\\"CVE-2020-27826\\"},{\\"cwe\\":\\"[CWE-377]\\",\\"severity\\":\\"HIGH\\",\\"name\\":\\"CVE-2021-20202\\"},{\\"cwe\\":\\"[CWE-330,CWE-341]\\",\\"severity\\":\\"CRITICAL\\",\\"name\\":\\"CVE-2020-1731\\"},{\\"cwe\\":\\"[CWE-80]\\",\\"severity\\":\\"MODERATE\\",\\"name\\":\\"CVE-2022-0225\\"},{\\"cwe\\":\\"[CWE-645]\\",\\"severity\\":\\"LOW\\",\\"name\\":\\"CVE-2024-1722\\"},{\\"cwe\\":\\"[CWE-200]\\",\\"severity\\":\\"MODERATE\\",\\"name\\":\\"CVE-2019-3868\\"},{\\"cwe\\":\\"[CWE-287]\\",\\"severity\\":\\"HIGH\\",\\"name\\":\\"CVE-2021-3632\\"},{\\"cwe\\":\\"[CWE-295]\\",\\"severity\\":\\"MODERATE\\",\\"name\\":\\"CVE-2020-35509\\"},{\\"cwe\\":\\"[CWE-79]\\",\\"severity\\":\\"MODERATE\\",\\"name\\":\\"UNKNOWN\\"},{\\"cwe\\":\\"[CWE-601,CWE-918]\\",\\"severity\\":\\"MODERATE\\",\\"name\\":\\"CVE-2020-10770\\"},{\\"cwe\\":\\"[CWE-20,CWE-352]\\",\\"severity\\":\\"HIGH\\",\\"name\\":\\"CVE-2019-10199\\"},{\\"cwe\\":\\"[CWE-347]\\",\\"severity\\":\\"MODERATE\\",\\"name\\":\\"CVE-2019-10201\\"},{\\"cwe\\":\\"[CWE-284,CWE-863]\\",\\"severity\\":\\"MODERATE\\",\\"name\\":\\"CVE-2023-0091\\"},{\\"cwe\\":\\"[CWE-295]\\",\\"severity\\":\\"MODERATE\\",\\"name\\":\\"CVE-2023-1664\\"},{\\"cwe\\":\\"[CWE-602]\\",\\"severity\\":\\"MODERATE\\",\\"name\\":\\"CVE-2017-12161\\"},{\\"cwe\\":\\"[CWE-116,CWE-20,CWE-79]\\",\\"severity\\":\\"CRITICAL\\",\\"name\\":\\"CVE-2021-20195\\"},{\\"cwe\\":\\"[CWE-22,CWE-552]\\",\\"severity\\":\\"MODERATE\\",\\"name\\":\\"CVE-2021-3856\\"},{\\"cwe\\":\\"[CWE-269,CWE-916]\\",\\"severity\\":\\"HIGH\\",\\"name\\":\\"CVE-2020-14389\\"},{\\"cwe\\":\\"[CWE-20]\\",\\"severity\\":\\"MODERATE\\",\\"name\\":\\"CVE-2021-3754\\"}]}'
             },
              }

In [9]:
str_to_json(example_node['n0']['value'])['cve']

[{'cwe': '[CWE-267]', 'severity': 'HIGH', 'name': 'CVE-2019-10170'},
 {'cwe': '[CWE-79]', 'severity': 'MODERATE', 'name': 'CVE-2022-0225'},
 {'cwe': '[CWE-79]', 'severity': 'MODERATE', 'name': 'CVE-2020-1697'},
 {'cwe': '[CWE-547,CWE-798]',
  'severity': 'CRITICAL',
  'name': 'CVE-2019-14837'},
 {'cwe': '[CWE-306]', 'severity': 'MODERATE', 'name': 'CVE-2021-20262'},
 {'cwe': '[CWE-1021]', 'severity': 'MODERATE', 'name': 'CVE-2020-1728'},
 {'cwe': '[CWE-285,CWE-287]', 'severity': 'HIGH', 'name': 'CVE-2018-14637'},
 {'cwe': '[CWE-276]', 'severity': 'LOW', 'name': 'UNKNOWN'},
 {'cwe': '[CWE-285]', 'severity': 'MODERATE', 'name': 'CVE-2020-10686'},
 {'cwe': '[CWE-20]', 'severity': 'HIGH', 'name': 'CVE-2020-1714'},
 {'cwe': '[CWE-287,CWE-841]', 'severity': 'MODERATE', 'name': 'UNKNOWN'},
 {'cwe': '[CWE-613]', 'severity': 'MODERATE', 'name': 'CVE-2020-1724'},
 {'cwe': '[CWE-835]', 'severity': 'MODERATE', 'name': 'CVE-2018-10912'},
 {'cwe': '[CWE-287]', 'severity': 'MODERATE', 'name': 'CVE-20

## process freshness value

In [7]:
# Filter nodes that have the 'severity' attribute
nodes_with_freshness = {node_id: node for node_id, node in nodes.items() if 'type' in node and node['type'] == "FRESHNESS" and str_to_json(node["value"])['freshness'] !={}}

In [8]:
for node_id, node in list(nodes_with_freshness.items())[:10]:
    print(node_id, str_to_json(node["value"]))
    print(node_id, node)

n29576356 {'freshness': {'numberMissedRelease': '7', 'outdatedTimeInMs': '3795765000'}}
n29576356 {'labels': ':AddedValue', 'id': 'com.softwaremill.sttp.client:core_sjs0.6_2.13:2.0.2:FRESHNESS', 'type': 'FRESHNESS', 'value': '{\\"freshness\\":{\\"numberMissedRelease\\":\\"7\\",\\"outdatedTimeInMs\\":\\"3795765000\\"}}'}
n29576358 {'freshness': {'numberMissedRelease': '2', 'outdatedTimeInMs': '11941344000'}}
n29576358 {'labels': ':AddedValue', 'id': 'com.ibeetl:act-sample:3.0.0-M6:FRESHNESS', 'type': 'FRESHNESS', 'value': '{\\"freshness\\":{\\"numberMissedRelease\\":\\"2\\",\\"outdatedTimeInMs\\":\\"11941344000\\"}}'}
n29576360 {'freshness': {'numberMissedRelease': '9', 'outdatedTimeInMs': '4685281000'}}
n29576360 {'labels': ':AddedValue', 'id': 'com.softwaremill.sttp.client:core_sjs0.6_2.13:2.0.0:FRESHNESS', 'type': 'FRESHNESS', 'value': '{\\"freshness\\":{\\"numberMissedRelease\\":\\"9\\",\\"outdatedTimeInMs\\":\\"4685281000\\"}}'}
n29576362 {'freshness': {'numberMissedRelease': '367'

## process popularity value

In [29]:
# Filter nodes that have the 'severity' attribute
nodes_with_popu = {node_id: node for node_id, node in nodes.items() if 'type' in node and node['type'] == "POPULARITY_1_YEAR" and node["value"] !='0'}

In [30]:
for node_id, node in list(nodes_with_popu.items())[:10]:
    print(node_id, node["value"])
    print(node_id, node)

n29576569 1
n29576569 {'labels': ':AddedValue', 'value': '1', 'id': 'io.gravitee.common:gravitee-common:3.1.0:POPULARITY_1_YEAR', 'type': 'POPULARITY_1_YEAR'}
n29576985 2
n29576985 {'labels': ':AddedValue', 'value': '2', 'id': 'org.thepalaceproject.audiobook:org.librarysimplified.audiobook.parser.api:11.0.0:POPULARITY_1_YEAR', 'type': 'POPULARITY_1_YEAR'}
n29577011 1
n29577011 {'labels': ':AddedValue', 'value': '1', 'id': 'com.lihaoyi:ammonite-compiler_2.12.11:2.5.11:POPULARITY_1_YEAR', 'type': 'POPULARITY_1_YEAR'}
n29577487 16
n29577487 {'labels': ':AddedValue', 'value': '16', 'id': 'org.apache.dubbo:dubbo-rpc-api:3.2.6:POPULARITY_1_YEAR', 'type': 'POPULARITY_1_YEAR'}
n29577779 2
n29577779 {'labels': ':AddedValue', 'value': '2', 'id': 'com.github.waikato:fcms-widgets:0.0.25:POPULARITY_1_YEAR', 'type': 'POPULARITY_1_YEAR'}
n29577815 2
n29577815 {'labels': ':AddedValue', 'value': '2', 'id': 'com.redislabs:jredisearch:2.2.0:POPULARITY_1_YEAR', 'type': 'POPULARITY_1_YEAR'}
n29577845 1
n29

## process speed value

In [31]:
# Filter nodes that have the 'severity' attribute
nodes_with_speed = {node_id: node for node_id, node in nodes.items() if 'type' in node and node['type'] == "SPEED" and node["value"] !='0'}

In [32]:
for node_id, node in list(nodes_with_speed.items())[:10]:
    print(node_id, node["value"])
    print(node_id, node)

n58494634 0.08070175438596491
n58494634 {'labels': ':AddedValue', 'id': 'se.fortnox.reactivewizard:reactivewizard-jaxrs:SPEED', 'type': 'SPEED', 'value': '0.08070175438596491'}
n58494635 0.08564476885644769
n58494635 {'labels': ':AddedValue', 'id': 'com.walmartlabs.concord.plugins.basic:http-tasks:SPEED', 'type': 'SPEED', 'value': '0.08564476885644769'}
n58494636 0.029411764705882353
n58494636 {'labels': ':AddedValue', 'id': 'cc.akkaha:asura-dubbo_2.12:SPEED', 'type': 'SPEED', 'value': '0.029411764705882353'}
n58494637 0.04967197750702906
n58494637 {'labels': ':AddedValue', 'id': 'org.wildfly.security:wildfly-elytron-jose-jwk:SPEED', 'type': 'SPEED', 'value': '0.04967197750702906'}
n58494638 0.3325480957989792
n58494638 {'labels': ':AddedValue', 'id': 'com.yahoo.vespa:container-messagebus:SPEED', 'type': 'SPEED', 'value': '0.3325480957989792'}
n58494639 0.014814814814814815
n58494639 {'labels': ':AddedValue', 'id': 'it.tidalwave.thesefoolishthings:it-tidalwave-thesefoolishthings-exampl